<a id='top'></a>

# Watershed metrics for all Huc12 subwatersheds that intersect AWC recorded streams
Iterate over AKSSF regions and identify all HUC12 sub-watersheds that intersect an AWC recorded stream. Identify the downstream-most/outlet catchment for each Huc12 from this pool and convert the polygon to INSIDE centroid point.  Calculate the distance to coastline as the straight line distance in Km from centroid point to NHD recorded coastline and export this as a feature class/table.  Next use the outlet catchments unique identifier to query the appropriate dataset and build watersheds for each outlet catchment.  Calculate watershed metrics listed in the covariate section and export final merged csv using the catchment unique identifier field "cat_ID_con" to link the metric back to the source catchment/HUC12.  Merge watersheds together and use to calculate covariates.
## Covariates
Covariates needed for prediction on AWC-HUC12 outlets are as follows:
### Summer Precipitation
To be calculated in R using the outlet catchment centroid point feature class exported during outlet identification process.
### Watershed Slope Metrics
Regional Slope grids created in AKSSF_merge_grids.ipynb script.
Run zonal statistics on slope grid using merged watershed as zone feature.
Field names and descriptions:
* **awc_huc12s_wtd_slope_mn = mean watershed slope**
* **awc_huc12s_wtd_slope_min = min watershed slope**
* **awc_huc12s_wtd_slope_max = max watershed slope**
* **awc_huc12s_wtd_slope_sd (or cv) = standard deviation of watershed slope**
### Watershed Percent North Aspect
Regional North grids created in AKSSF_merge_grids.ipynb scripts.
North = aspects from 315-45 degrees and calculate the percentage of land area facing north for each watershed. Run tabulate area on north grid using merged watershed as zone feature and calculate percentage from area.
Field names and descriptions:
* **awc_huc12s_north_wtd = percent watershed with north aspect**
### Watershed Percent Lake Cover
Lakes feature classes for each network datatype (NHDPlus vs TauDEM) stored in AKSSF hydrography database on the T:
Calculate percentage of watershed that is covered by lakes/ponds using tabulate interesection between lake features and watersheds.
Field names and descriptions:
* **awc_huc12s_wtd_lake_per = percent watershed covered by lakes**
### Watershed Percent Glacier Cover
Use input glacier fc (from previous covariate calculations) stored in regional gdbs an calculate percent of watershed with glacial coverage using tabulate intersection between lake features and watersheds.
Field names and descriptions:
* **awc_huc12s_wtd_glac_per = percent watershed covered by glaciers**
### Watershed LCLD
LCLD rasters created in AKSSF_MODIS_lcld_ipynb.
Iterate over LCLD input rasters to produce yearly means for watersheds using zonal statistics.
Field names and descriptions:
* **awc_huc12s_wtd_lcld_mn_YYYY = mean lcld**


## Import modules
Set initial environments and import modules
Print system paths

In [1]:
import os, arcpy, sys,datetime, traceback

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())
outdir = os.path.dirname(os.getcwd())
print(f'CSV table output directory set to {outdir}')

imports complete
----------------------------------------------------------------------------------------------------
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\landcover', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin', 'C:\\Users\\dwmerrigan\\AppDa

## Functions
Define any functions that will be used

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput

# Getnull rows from numpy array
def getnull(cat_ID_con):
    nullRows = []
    nullRows.append(cat_ID_con)
    return True

#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

## Section 1
### Set input datasets, output locations, and scratch workspaces
User to input paths for necessary input data and output locations
Scratch workspaces and output workspaces will be automatically created if they do not already exist.

In [3]:
# Get user inputs
# Used to format user inputs
inputDict = {"'":"",'"':""}

# Specify path to nhdPlus lakes
while True:
    try:
        userinput7 = replace_all((input('Input path to NHDPlus lakes feature class.\nHydrography database on T: has copy\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond_alb') or 'D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond_alb'),inputDict)
        if not arcpy.Exists(userinput7):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            nhd_lakes_fc = userinput7
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()

print(f'NHDPlus lakes set to {nhd_lakes_fc}\n {"-"*100}')

# Specify path to nhd lakes for tau regions
while True:
    try:
        userinput8 = replace_all((input('Input path to NHD_H_Alaska_State_GDB lakes feature class.\nHydrography database on T: has copy\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds_alb') or 'D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds_alb'),inputDict)
        if not arcpy.Exists(userinput8):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            tau_lakes_fc = userinput8
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'NHD_H_Alaska lakes for TauDEM regions set to {nhd_lakes_fc}\n {"-"*100}')

# Specify path to coastline fc
while True:
    try:
        userinput9 = replace_all((input('Input path to coastline fc.\nHydrography database on T: has copy\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\NHD_H_Alaska_Coastline_alb') or 'D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\NHD_H_Alaska_Coastline_alb'),inputDict)
        if not arcpy.Exists(userinput8):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            coast = userinput9
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'Coastline for all regions set to {coast}\n {"-"*100}')

# Specify path to AKSSF parent directory
while True:
    try:
        userinput = replace_all((input('Input AKSSF parent directory containing regional sub-folders.\nLeave blank and hit enter to use the default location.\nDefault = D:\\GIS\\AKSSF\\') or 'D:\\GIS\\AKSSF'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            data_dir = userinput
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()

print(f'AKSSF parent directory set to {data_dir}\n {"-"*100}')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events = userinput2
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AWC events feature class set to {awc_events}\n {"-"*100}')

# Enter output destination  - to create working folders and gdbs
while True:
    try:
        userinput3 = replace_all((input('Input path to create working folders.\nLeave blank and hit enter to use the default location.\nDefault = D:\\GIS\\') or 'D:\\GIS\\'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            temp_path = userinput3
            print(f'Output locations will be created at {temp_path}\n {"-"*100}')
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()


# Path to lcld rasters
lcld_folder = r'D:\\Basedata\\LCLD_rasters_archive'
# Enter output destination  - to create working folders and gdbs
while True:
    try:
        userinput4 = replace_all((input('Input path to LCLD raster parent folder.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\LCLD_rasters_archive\\') or 'D:\\Basedata\\LCLD_rasters_archive'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            lcld_folder = userinput4
            print(f'LCLD subfolders located at {lcld_folder}\n {"-"*100}')
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()

## Create working output location to store intermediate data
dirname = 'AKSSF_awcHuc12_cv'
tempgdbname = 'AKSSF_awcHuc12_cv.gdb'
temp_dir = os.path.join(temp_path, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print(f'Working Folder already created {temp_dir}\n {"-"*100}')

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print (f'Output location already exists {outcheck}\n {"-"*100}')
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print(f'Creating output GDB\n {"-"*100}')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print (f'Output geodatabase created at {outcheck}\n {"-"*100}')
    outgdb = tempgdb.getOutput(0)


NHDPlus lakes set to D:\Basedata\AKSSF_Basedata\AKSSF_Basedata.gdb\AKSSF_NHDPlus_LakePond_alb
 ----------------------------------------------------------------------------------------------------
NHD_H_Alaska lakes for TauDEM regions set to D:\Basedata\AKSSF_Basedata\AKSSF_Basedata.gdb\AKSSF_NHDPlus_LakePond_alb
 ----------------------------------------------------------------------------------------------------
Coastline for all regions set to D:\Basedata\AKSSF_Basedata\AKSSF_Basedata.gdb\NHD_H_Alaska_Coastline_alb
 ----------------------------------------------------------------------------------------------------
AKSSF parent directory set to D:\GIS\AKSSF
 ----------------------------------------------------------------------------------------------------
AWC events feature class set to D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\awcEventArcs
 ----------------------------------------------------------------------------------------------------
Output locations will be created at D:\GI

## Section 1.1
### Set and create local copies of additional input data


In [4]:
import arcpy
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

nhdplusfol = []
tahuc12=[]

# Create and set HUC12 data if it does not already exist
nhdplushucs = os.path.join(outgdb, 'NHDPlusHUC12')
tauhucs = os.path.join(outgdb, 'NHD_H_HUC12')

if not arcpy.Exists(tauhucs):
    print(f'Huc12 data for Tau Regions not yet created')
    #Enter path to WBDHU12 from NHD_H gdb
    while True:
        try:
            userinput6 = replace_all((input('Input path to source WBDHU12 for state of Alaska.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\NHD_H_Alaska_State_GDB.gdb\\WBD\\WBDHU12') or 'D:\\Basedata\\NHD_H_Alaska_State_GDB.gdb\\WBD\\WBDHU12'),inputDict)
            if not arcpy.Exists(userinput6):
                print('Path specified does not exist!\nPlease re-enter a valid path')
                continue
            else:
                tauhuc12 = userinput6
                arcpy.CopyFeatures_management(tauhuc12,tauhucs)
                print(f'WBD Huc12  copied to {tauhucs}\n {"-"*100}')
                break
        except KeyboardInterrupt:
            print('interrupted!')
            sys.exit()

else:
    print(f'Tau Region Hucs {tauhucs} located and exists = {arcpy.Exists(tauhucs)}')

if not arcpy.Exists(nhdplushucs):
    print(f'Huc12 data for NHDPlus Regions not yet created')
    #Enter NHDplus data folder
    while True:
        try:
            userinput5 = replace_all((input('Input path to source NHDPlus parent folder.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\NHDPlus') or 'D:\\Basedata\\NHDPlus'),inputDict)
            if not arcpy.Exists(userinput5):
                print('Path specified does not exist!\nPlease re-enter a valid path')
                continue
            else:
                nhdplusfol = userinput5
                print(f'NHD HUC12 will be copied to {nhdplushucs}\n {"-"*100}')
                hucs = []
                walk = arcpy.da.Walk(nhdplusfol, datatype="FeatureClass", type="Polygon")

                for dirpath, dirnames, filenames in walk:
                    for filename in filenames:
                        if filename == 'WBDHU12':
                            hucs.append(os.path.join(dirpath, filename))
                arcpy.Merge_management(hucs,nhdplushucs,'','ADD_SOURCE_INFO')
                break
        except KeyboardInterrupt:
            print('interrupted!')
            sys.exit()
else:
    print(f'NHDPlus Hucs {nhdplushucs} located and exists = {arcpy.Exists(nhdplushucs)}')



Huc12 data for Tau Regions not yet created
WBD Huc12  copied to D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\NHD_H_HUC12
 ----------------------------------------------------------------------------------------------------
Huc12 data for NHDPlus Regions not yet created
NHD HUC12 will be copied to D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\NHDPlusHUC12
 ----------------------------------------------------------------------------------------------------


## Section 2
## Identify outlet catchments by region
Identify downstream-most catchment for each Huc 12
 * Select by location and select catchment with most us contributing area
    * NHDPlus
        * Use update cursor to join TotalDrainageAreaSqKm from vaa table to catchment
        * Find max value from selection and save as outlet catchment for that HUC12
    * TauDEM
        * DSContArea - Drainage area at the downstream end of the link. Generally this is one grid cell upstream of the downstream end because the drainage area at the downstream end grid cell includes the area of the stream being joined.
 * Generate Centroid point and append to centroid dataset
    * Retain cat_id and Huc12-id
 * Append to HUC12 catchment dataset

In [5]:
import arcpy, time, os, datetime, operator

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces()

# Dictionaries and lists
nhdplusoutlets = []
tauoutlets = []
nhdplusawccatouts = []
tauawccatouts = []
dist2CoastDict = {}
vaaDict = {}
strDict = {}
catsDict = {}
huc12Dict = {}
nhdidDict = {}
tauidDict = {}
tauhuc12Dict = {}

# Separate data by source type
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
# rois = nhdplus_dat + tauDem_dat

# Or comment above and specify below specific subset
regions = ['D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River' ,'D:\\GIS\\AKSSF\\Prince_William_Sound']

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    roi = os.path.basename(region)
    print(roi)
    if roi in nhdplus_dat:
        # Start roi time
        roi_start = time.time()
        hucs = nhdplushucs
        catsList = []
        outletList = []
        print(f'{roi} using data from {region} folder')
        # Set workspace to region folder
        arcpy.env.workspace = region
        gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
        print(f'GDB {gdb}')
        sourcegdb = gdb[0]
        walk = arcpy.da.Walk(sourcegdb, datatype = ['FeatureClass','Table'])
        for dirpath, dirnames, filenames in walk:
            for filename in filenames:
                if filename == 'cats_merge':
                    cats  = os.path.join(dirpath, filename)
                    append_value(catsDict,roi,cats)
                elif filename == 'vaa_merge':
                    vaas = os.path.join(dirpath, filename)
                    append_value(vaaDict, roi, vaas)
        #Output names and paths
        outletcatsname = roi + '_AwcHuc12_cats_outlets'
        outcatspath = os.path.join(outgdb,outletcatsname)
        outcatspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets')
        outletcatptsname = roi + '_AwcHuc12_cats_outlets_pts'
        outcatptspath = os.path.join(outgdb,outletcatptsname)
        outcatptspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets_pts')

        if not arcpy.Exists(outcatspath):
            # Build Value dictionary to relate NHDPlus id to contributing area
            fields = ['NHDPlusID','TotDASqKm']
            fields2 = fields + ['cat_ID_con']
            valueDict = {int(r[0]):(r[1]) for r in arcpy.da.SearchCursor(vaas, fields)}
            where_clause=f'"MERGE_SRC" LIKE \'%{roi}%\''
            print(f'where_clause = {where_clause}')
            huclayer = arcpy.MakeFeatureLayer_management(hucs,'huclayer',where_clause = where_clause)
            print(f'{arcpy.GetCount_management(huclayer)} huc12s in {roi}')
            print(('*'*100))
            hucselect = arcpy.SelectLayerByLocation_management(huclayer,'INTERSECT',awc_events,'','SUBSET_SELECTION')
            print(('*'*100))
            print(f'{arcpy.GetCount_management(hucselect)} Huc12s in {roi} intersect awc events input')
            print(('*'*100))
            hucFields = [f for f in arcpy.ListFields(hucselect)]
            vcount =1
            with arcpy.da.SearchCursor(hucselect,['HUC12','SHAPE@']) as cur:
                for row in cur:
                    print(f'Processing HUC {row[0]}')
                    inhuc = row[1]
                    cat_layer = arcpy.MakeFeatureLayer_management(cats,'cat_layer')
                    # Select by location using awc and huc 12
                    arcpy.SelectLayerByLocation_management(cat_layer,'HAVE_THEIR_CENTER_IN',inhuc,'','NEW_SELECTION')
                    print(f'{vcount}. Finding outlet for HUC {row[0]} out of {arcpy.GetCount_management(cat_layer)} catchments.\n{("*" * 60)}')
                    catList = [r[0] for r in arcpy.da.SearchCursor(cat_layer, 'NHDPlusID')]
                    intersect = list(set(catList).intersection(valueDict))
                    catDict = {int(i):(valueDict[i]) for i in intersect}
                    # Find Catchment with max drainage area
                    outcatch = max(catDict.items(), key = operator.itemgetter(1))[0]
                    cidcon = roi + '_' + str(int(outcatch))
                    append_value(dist2CoastDict,cidcon,row[0])
                    append_value(huc12Dict, row[0], [int(outcatch),roi,valueDict[int(outcatch)]])
                    append_value(nhdidDict,int(outcatch),[roi,row[0], valueDict[int(outcatch)]])
                    outletList.append(int(outcatch))
                    vcount+=1
                del(row)
            del(cur)

            outlet_cats = arcpy.MakeFeatureLayer_management(cats,'outlet_cats')
            out_expression ='"NHDPlusID" IN ' + str(tuple(outletList))
            #print(out_expression)
            outlet_cats_select = arcpy.SelectLayerByAttribute_management(outlet_cats,'NEW_SELECTION', out_expression)
            print(f'Creating copy of {arcpy.GetCount_management(outlet_cats)} outlet catchments for Region {roi} at {outcatspath}')
            print(('*'*100))

            # Copy outputs
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,outgdb,outletcatsname)
            arcpy.FeatureToPoint_management(outcatspath, outcatptspath, 'INSIDE')
            # Create Copies to akssf data_dir regional gdbs also
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,sourcegdb,'awc_huc12_catchment_outlets')
            arcpy.FeatureToPoint_management(outcatspath2, outcatptspath2, 'INSIDE')
            nhdplusoutlets.append(outcatptspath)
            nhdplusawccatouts.append(outcatspath)
            # Add total drainage km from value dict to feature classes and cat_ID_con from regDict
            upfcs = [outcatspath, outcatptspath,outcatptspath2,outcatptspath2]
            for upfc in upfcs:
                arcpy.AddField_management(upfc,fields[1],'TEXT')
                arcpy.AddField_management(upfc,fields2[2],'TEXT')
                with arcpy.da.UpdateCursor(upfc,fields2) as cur:
                    for row in cur:
                        row[1] = valueDict[row[0]]
                        row[2] = roi + '_' + str(int(row[0]))
                        cur.updateRow(row)
                    del(row)
                del(cur)

            # End roi time
            roi_stop = time.time()
            roi_time = int (roi_stop - roi_start)
            print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=roi_time)})')
            print(f'{"*"*60}')
        else:
            print(f'Catchments for {roi} already created at {outcatspath2}')

    elif roi in tauDem_dat:
        # Start roi time
        roi_start = time.time()
        hucs = tauhucs
        catsList = []
        outletList = []
        print(f'{roi} using data from {region} folder')
        # Set workspace to region folder
        arcpy.env.workspace = region
        gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
        sourcegdb = gdb[0]
        walk = arcpy.da.Walk(sourcegdb, datatype = ['FeatureClass','Table'])
        for dirpath, dirnames, filenames in walk:
            for filename in filenames:
                if filename == 'cats_merge':
                    cats  = os.path.join(dirpath, filename)
                    append_value(catsDict,roi,cats)
                elif filename == 'streams_merge':
                    streams = os.path.join(dirpath, filename)
                    append_value(strDict, roi, streams)

        #Output names and paths
        outletcatsname = roi + '_TauAwcH12_cats_outlets'
        outcatspath = os.path.join(outgdb,outletcatsname)
        outcatspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets')
        outletcatptsname = roi + '_TauAwcH12_cats_outlets_pts'
        outcatptspath = os.path.join(outgdb,outletcatptsname)
        outcatptspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets_pts')
        print(('-'*100),'\n')
        if not arcpy.Exists(outcatspath):
            # Build Value dictionary to relate NHDPlus id to contributing area
            fields = ['LINKNO','DSContArea']
            fields2 = fields + ['cat_ID_con']
            fields3 = ['gridcode','DSContArea','cat_ID_con']
            valueDict = {int(r[0]):(r[1]) for r in arcpy.da.SearchCursor(streams, fields)}
            huclayer = arcpy.MakeFeatureLayer_management(hucs,'huclayer')
            hucselect_reg = arcpy.SelectLayerByLocation_management(huclayer,'INTERSECT',streams,'','NEW_SELECTION')
            print(f'{arcpy.GetCount_management(huclayer)} huc12s in {roi}')
            print(('*'*100))
            hucselect = arcpy.SelectLayerByLocation_management(hucselect_reg,'INTERSECT',awc_events,'','SUBSET_SELECTION')
            print(('*'*100))
            print(f'{arcpy.GetCount_management(hucselect)} Huc12s in {roi} intersect awc events input')
            print(('*'*100))
            hucFields = [f for f in arcpy.ListFields(hucselect)]
            vcount =1
            with arcpy.da.SearchCursor(hucselect,['HUC12','SHAPE@']) as cur:
                for row in cur:
                    print(f'Processing HUC {row[0]}')
                    inhuc = row[1]
                    cat_layer = arcpy.MakeFeatureLayer_management(cats,'cat_layer')
                    # Select by location using awc and huc 12
                    arcpy.SelectLayerByLocation_management(cat_layer,'HAVE_THEIR_CENTER_IN',inhuc,'','NEW_SELECTION')
                    print(f'{vcount}. Finding outlet for HUC {row[0]} out of {arcpy.GetCount_management(cat_layer)} catchments.\n{("*" * 60)}')
                    catList = [r[0] for r in arcpy.da.SearchCursor(cat_layer, 'gridcode')]
                    intersect = list(set(catList).intersection(valueDict))
                    catDict = {int(i):(valueDict[i]) for i in intersect}
                    # Find Catchment with max drainage area
                    outcatch = max(catDict.items(), key = operator.itemgetter(1))[0]
                    cidcon = roi +'_'+ str(int(outcatch))
                    append_value(dist2CoastDict,cidcon,row[0])
                    append_value(tauhuc12Dict, row[0], [int(outcatch),roi,valueDict[int(outcatch)]])
                    append_value(tauidDict,int(outcatch),[roi,row[0], valueDict[int(outcatch)]])
                    outletList.append(int(outcatch))
                    vcount+=1
                del(row)
            del(cur)
            outlet_cats = arcpy.MakeFeatureLayer_management(cats,'outlet_cats')
            out_expression ='"gridcode" IN ' + str(tuple(outletList))
            #print(out_expression)
            outlet_cats_select = arcpy.SelectLayerByAttribute_management(outlet_cats,'NEW_SELECTION', out_expression)
            print(f'Creating copy of {arcpy.GetCount_management(outlet_cats)} outlet catchments for Region {roi} at {outcatspath}')
            print(('*'*100))

            # Copy outputs
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,outgdb,outletcatsname)
            arcpy.FeatureToPoint_management(outcatspath, outcatptspath, 'INSIDE')
            # Create Copies to akssf data_dir regional gdbs also
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,sourcegdb,'awc_huc12_catchment_outlets')
            arcpy.FeatureToPoint_management(outcatspath2, outcatptspath2, 'INSIDE')
            tauoutlets.append(outcatptspath)
            tauawccatouts.append(outcatspath)
            # Add total drainage km from value dict to feature classes and cat_ID_con from regDict
            upfcs = [outcatspath, outcatptspath,outcatptspath2,outcatptspath2]
            for upfc in upfcs:
                arcpy.AddField_management(upfc,fields[1],'TEXT')
                arcpy.AddField_management(upfc,fields2[2],'TEXT')
                with arcpy.da.UpdateCursor(upfc,fields3) as cur:
                    for row in cur:
                        row[1] = valueDict[row[0]]
                        row[2] = roi + '_' + str(int(row[0]))
                        cur.updateRow(row)
                    del(row)
                del(cur)

            # End roi time
            roi_stop = time.time()
            roi_time = int (roi_stop - roi_start)
            print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=roi_time)})')
            print(f'{"*"*60}')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
print(f'Process complete')

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')

Cook_Inlet
Cook_Inlet using data from D:\GIS\AKSSF\Cook_Inlet folder
GDB ['D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb']
where_clause = "MERGE_SRC" LIKE '%Cook_Inlet%'
1058 huc12s in Cook_Inlet
****************************************************************************************************
****************************************************************************************************
661 Huc12s in Cook_Inlet intersect awc events input
****************************************************************************************************
Processing HUC 190202020501
1. Finding outlet for HUC 190202020501 out of 95 catchments.
************************************************************
Processing HUC 190202020503
2. Finding outlet for HUC 190202020503 out of 354 catchments.
************************************************************
Processing HUC 190202020507
3. Finding outlet for HUC 190202020507 out of 218 catchments.
************************************************************


## Section 2.1
### Merge all outlet points together and calculate distance to coastline
This chunk is dependent on the data dictionaries created in Step 2
Calculate Distance to Coast from outlet catchment point to the nearest coastline as a straight line distance
 * Generate near table and export as seperate csv

NHDPlus Section


In [6]:
import arcpy, datetime
import numpy as pd

# Input path to coastline
coast = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\NHD_H_Alaska_Coastline_alb"

# Merge all catchment outlet centroids together
nhdoutletsname = 'AKSSF_NHDPlus_awcHuc12_outlet_cats_points'
nhdoutletspath = os.path.join(outgdb, nhdoutletsname)

if not arcpy.Exists(nhdoutletspath):
    all_nhd_outlet_pts = arcpy.Merge_management(nhdplusoutlets,nhdoutletspath)
    # Start timing function
    start = datetime.datetime.now()
    print(f'Getting distance to coast {datetime.datetime.now()}...')
    arcpy.analysis.Near(all_nhd_outlet_pts, coast, None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_DIST")
    arcpy.AlterField_management(all_nhd_outlet_pts,'NEAR_DIST','dist_catch_coast_km','dist_catch_coast_km' )
    arcpy.AddField_management(all_nhd_outlet_pts,'HUC12','TEXT')

    # Convert distance in meters to km
    with arcpy.da.UpdateCursor(all_nhd_outlet_pts,['dist_catch_coast_km','NHDPlusID','HUC12']) as cur:
        for row in cur:
            row[0] = row[0] * 0.001
            row[2] = nhdidDict[row[1]][1]
            cur.updateRow(row)
        del(row)
    del(cur)
    print(f'Process complete')
else:
    print(f'Outlet points already created at {nhdoutletspath}')


Getting distance to coast 2022-02-14 18:30:25.259429...
Process complete


TauDEM Section


In [7]:
import arcpy, datetime
import numpy as pd

# Input path to coastline
coast = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\NHD_H_Alaska_Coastline_alb"

# Merge all catchment outlet centroids together
tauoutname = 'AKSSF_TauDEM_awcHuc12_outlet_cats_points'
tauoutpath = os.path.join(outgdb, tauoutname)

if not arcpy.Exists(tauoutpath):
    all_tau_outpts = arcpy.Merge_management(tauoutlets,tauoutpath)
    # Start timing function
    start = datetime.datetime.now()
    print(f'Getting distance to coast {datetime.datetime.now()}...')
    arcpy.analysis.Near(all_tau_outpts, coast, None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_DIST")
    arcpy.AlterField_management(all_tau_outpts,'NEAR_DIST','dist_catch_coast_km','dist_catch_coast_km' )
    arcpy.AddField_management(all_tau_outpts,'HUC12','TEXT')
    arcpy.AddField_management(all_tau_outpts,'DSContAreaSqKM','DOUBLE')

    # Convert distance in meters to km
    with arcpy.da.UpdateCursor(all_tau_outpts,['dist_catch_coast_km','gridcode','HUC12','DSContArea','DSContAreaSqKM']) as cur:
        for row in cur:
            row[0] = row[0] * 0.001
            row[2] = tauidDict[row[1]][1]
            row[4] = int(row[3])/1000000 #convert sq meters to sq km
            cur.updateRow(row)
        del(row)
    del(cur)
    print(f'Process complete')
else:
    print(f'Outlet points already created at {tauoutpath}')

Getting distance to coast 2022-02-14 18:30:59.543958...
Process complete



## Merge NHD and Tau points together and export as CSV

In [8]:
# NHDPoints
nhdoutletsname = 'AKSSF_NHDPlus_awcHuc12_outlet_cats_points'
nhdoutletspath = os.path.join(outgdb, nhdoutletsname)

# Taupoints
tauoutname = 'AKSSF_TauDEM_awcHuc12_outlet_cats_points'
tauoutpath = os.path.join(outgdb, tauoutname)

# All points
catpointsname = 'AKSSF_awcHuc12_outlet_cats_points'
catpointspath = os.path.join(outgdb, catpointsname)

# Create FieldMappings object to manage merge output fields
out_fms = arcpy.FieldMappings()

# Add all fields from both point fcs
out_fms.addTable(nhdoutletspath)
out_fms.addTable(tauoutpath)

# Add input fields
out_fm_dsdrain = arcpy.FieldMap()
out_fm_dsdrain.addInputField(nhdoutletspath,'TotDASqKm')
out_fm_dsdrain.addInputField(tauoutpath,'DSContAreaSqKM')

# Set name of new output field "Street_Name"
dsdrain = out_fm_dsdrain.outputField
dsdrain.name = "DsContAreaSqKm"
out_fm_dsdrain.outputField = dsdrain

# add to field mappings
out_fms.addFieldMap(out_fm_dsdrain)

for field in out_fms.fields:
    if field.name not in ['cat_ID_con', 'DsContAreaSqKm','dist_catch_coast_km', 'HUC12']:
        out_fms.removeFieldMap(out_fms.findFieldMapIndex(field.name))

#if not arcpy.Exists(catpointspath):
addSourceInfo = "ADD_SOURCE_INFO"
cats_outlets = arcpy.Merge_management([nhdoutletspath,tauoutpath],
                                      catpointspath,
                                      out_fms,
                                      addSourceInfo)
print(f'Merging all available catchment outlet points')
#else:
#    print(f'AKSSF AWC Catchment outlets already identified and exported to {catpointspath}')


Merging all available catchment outlet points


### Convert to df and examine

In [9]:
import numpy
import pandas as pd

# Make catchment points df
cat_df = pd.DataFrame()
cat_field_list = []

for field in arcpy.ListFields(catpointspath):
    print(field.name)
    cat_field_list.append(field.name)
cat_arr = arcpy.da.TableToNumPyArray(catpointspath, ['cat_ID_con','dist_catch_coast_km','DsContAreaSqKm','HUC12'])
cat_df = pd.DataFrame(cat_arr)
cat_df = cat_df.set_index('cat_ID_con')
cat_df

OBJECTID
Shape
cat_ID_con
dist_catch_coast_km
HUC12
DsContAreaSqKm
MERGE_SRC


,dist_catch_coast_km,DsContAreaSqKm,HUC12
cat_ID_con,,,
Cook_Inlet_75004200000901,21.633384,36.96257508,190202020501
Cook_Inlet_75004200001724,17.599013,170.81217492,190202020503
Cook_Inlet_75004200001726,0.128861,570.1149751,190202020508
Cook_Inlet_75004200001493,0.147499,9.6406,190202020303
Cook_Inlet_75004200004105,0.137137,14.02302496,190202020102
...,...,...,...
Prince_William_Sound_91741,17.543384,572.85728,190202010906
Prince_William_Sound_91751,17.338805,574.793792,190202010905
Prince_William_Sound_91921,12.177571,739.347904,190202010907


### Export csv of outlet points for NHDPlus regions


In [10]:
import os
#Export CSV to read into R
catpts_outname = 'AKSSF_awcHuc12_dist_catch_coast_km.csv'
outlets_csv = os.path.join(outdir,catpts_outname)
if not arcpy.Exists(outlets_csv):
    arcpy.da.NumPyArrayToTable(cat_arr,outlets_csv)
    print('CSV export complete')
else:
    print(f'Csv of catchment outlet points already exported to {outlets_csv}')
print('----------')

Csv of catchment outlet points already exported to C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_awcHuc12_dist_catch_coast_km.csv
----------


## Section 3
### Watersheds
Generate Watersheds
* If watersheds have already been created there is no need to run this section again in order for subsequent process to run.

In [11]:
# NHDPLUS Watersheds

import arcpy, time, datetime, os
import pandas as pd
from functools import reduce
import arcpy, time, os, datetime, operator

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces()
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

wtdDict = {}

# Separate data by source type
nhdplus_dat = ['Cook_Inlet','Copper_River']
# Limit to Cook inlet for testing
regions = ['D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River' ,'D:\\GIS\\AKSSF\\Prince_William_Sound']
#regions = ['D:\\GIS\\AKSSF\\Prince_William_Sound']

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    reg_start = time.time()
    roi = os.path.basename(region)
    print(roi)
    if roi in nhdplus_dat:
        try:
            wtdList = []
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            ingdb = gdb[0]
            # set inputs
            vaa = os.path.join(ingdb, "vaa_merge")
            cats = os.path.join(ingdb, "cats_merge")
            streams = os.path.join(ingdb, "NHDFlowline_merge")
            outcats = os.path.join(ingdb, "awc_huc12_catchment_outlets")
            # Create list of nhdplus ids for outlet catchments
            idList = [int(row[0]) for row in arcpy.da.SearchCursor(outcats,'NHDPlusID')]
            #Make test list of few small catchments
            #idList = [75004400004166,75004400004344, 75004400010328]
            # Get list of index names for cats merge and add index if not already created
            index_names = [i.name for i in arcpy.ListIndexes(cats)]
            print(index_names)
            if 'NHDPlusID_index' not in index_names:
                print (f'Creating index for {cats}')
                arcpy.AddIndex_management(cats,'NHDPlusID','NHDPlusID_index')
            else:
                print(f'{cats} Indexed')

            #watersheds feature dataset for storing fcs
            fdatname = roi + '_Watersheds'
            fdat = os.path.join(outgdb,fdatname)
            if not arcpy.Exists(fdat):
                arcpy.management.CreateFeatureDataset(outgdb, fdatname, sr)
            else:
                print(f'{fdat} exists for {roi}')

            vaa_df1 = pd.DataFrame(arcpy.da.TableToNumPyArray(vaa, ("NHDPlusID", "FromNode", "ToNode", "StartFlag")))
            stream_df = pd.DataFrame(arcpy.da.TableToNumPyArray(streams, ("NHDPlusID", "FType")))
            dfs = [vaa_df1, stream_df]
            vaa_df = reduce(lambda left,right: pd.merge(left,right,on='NHDPlusID',how="outer"), dfs)
            # remove pipelines
            vaa_df = vaa_df[(vaa_df['FType'] != 428 )]
            vaa_df

            c=1
            for id in idList:
                iteration_start = time.time()
                print(f'{c}. Starting watershed for HUC {str(id)} ({(len(idList) - c)} remaining)')
                rec = [id]
                up_ids = []
                up_ids.append(rec)
                rec_len = len(rec)
                hws_sum = 0

                while rec_len != hws_sum:
                    fromnode = vaa_df.loc[vaa_df["NHDPlusID"].isin(rec), "FromNode"]
                    rec = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "NHDPlusID"]
                    rec_len = len(rec)
                    rec_hws = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "StartFlag"]
                    hws_sum = sum(rec_hws)
                    up_ids.append(rec)
                #up_ids is a list with more than numbers, use extend to only keep numeric nhdplusids
                newup_ids = []
                for x in up_ids:
                    newup_ids.extend(x)

                tempLayer = "catsLyr"
                expression = '"NHDPlusID" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
                arcpy.MakeFeatureLayer_management(cats, tempLayer, where_clause=expression)
                outdis = "memory/wtd_" + str(round(id))
                outwtd = os.path.join(fdat,f'{roi}_wtd_{str(int(id))}')
                dis = arcpy.Dissolve_management(tempLayer, outdis)
                watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
                wtdList.append(outwtd)
                append_value(wtdDict,roi,outwtd)

                # Stop iteration timer
                iteration_stop = time.time()
                iter_time = int (iteration_stop - iteration_start)
                print(f'Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
                print(f'{"*"*60}')
                c+=1

            wtd_merge = arcpy.Merge_management(wtdList, os.path.join(ingdb,'awc_huc12_wtds_merge'),'','ADD_SOURCE_INFO')
            arcpy.AddField_management(wtd_merge,'cat_ID_con','TEXT')
            arcpy.AddField_management(wtd_merge,'cat_ID','DOUBLE')
            arcpy.AddField_management(wtd_merge,'cat_ID_txt','TEXT')
            arcpy.AddField_management(wtd_merge,'NHDPlusID','DOUBLE')
            with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','NHDPlusID','cat_ID_con','cat_ID','cat_ID_txt']) as cur:
                for row in cur:
                    # Pull nhdplus id from merge source and calculate fields
                    nhdplusid= int(row[0].split('_')[-1])
                    row[1] = nhdplusid
                    row[2] = roi + '_' + str(nhdplusid)
                    row[3] = nhdplusid
                    row[4] = str(nhdplusid)
                    cur.updateRow(row)
                del(row)
            del(cur)
            arcpy.CopyFeatures_management(wtd_merge,os.path.join(outgdb,f'{roi}_AwcHuc12_wtds_merge' ))

            # Stop iteration timer
            reg_stop = time.time()
            reg_time = int (reg_stop - reg_start)
            print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=reg_time)})')
            print(f'{"*"*100}')

        except:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])

    elif roi in tauDem_dat:
        try:
            reg_start = time.time()
            wtdList = []
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            ingdb = gdb[0]
            # set inputs
            cats = os.path.join(ingdb, "cats_merge")
            streams = os.path.join(ingdb, "streams_merge")
            outcats = os.path.join(ingdb, "awc_huc12_catchment_outlets")
            # Create list of nhdplus ids for outlet catchments
            idList = [int(row[0]) for row in arcpy.da.SearchCursor(outcats,'gridcode')]
            index_names = [i.name for i in arcpy.ListIndexes(cats)]
            if 'catid_index' not in index_names:
                print (f'Creating index for {cats}')
                arcpy.AddIndex_management(cats, "catID", "catid_index")
            else:
                print(f'{cats} Indexed')
            #watersheds feature dataset for storing fcs
            fdatname = roi + '_Watersheds'
            fdat = os.path.join(outgdb,fdatname)
            if not arcpy.Exists(fdat):
                arcpy.management.CreateFeatureDataset(outgdb, fdatname, sr)
            else:
                print(f'{fdat} exists for {roi}')
            fields = arcpy.ListFields(streams)
            str_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(streams, ("LINKNO", "USLINKNO1", "USLINKNO2")))
            hws_codes = [-1]

            # Generate watersheds
            c=1
            for id in idList:
                iteration_start = time.time()
                print(f'{c}. Starting watershed for HUC {str(id)} ({(len(idList) - c)} remaining)')
                rec = [id]
                up_ids = []
                sum_rec = sum(rec)
                while(sum_rec > 0):
                    up_ids.append(rec)
                    rec = str_df.loc[str_df["LINKNO"].isin(rec), ("USLINKNO1", "USLINKNO2")]
                    rec = pd.concat([rec['USLINKNO1'], rec['USLINKNO2']])
                    sum_rec = sum(rec)
                # up_ids is a list with more than numbers, use extend to only keep numeric nhdplusids
                newup_ids = []
                for x in up_ids:
                    newup_ids.extend(x)

                tempLayer = "catsLyr"
                expression = '"gridcode" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
                arcpy.MakeFeatureLayer_management(cats, tempLayer)
                arcpy.management.SelectLayerByAttribute(tempLayer, "NEW_SELECTION", expression, None)
                print("Starting dissolve")
                outdis = "memory/wtd_" + str(round(id))
                outwtd = os.path.join(fdat,f'{roi}_wtd_{str(int(id))}')
                dis = arcpy.Dissolve_management(tempLayer, outdis)
                watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
                wtdList.append(watershed)
                append_value(wtdDict,roi,outwtd)

                # Stop iteration timer
                iteration_stop = time.time()
                iter_time = int (iteration_stop - iteration_start)
                print(f'Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
                print(f'{"*"*60}')
                c+=1

            wtd_merge = arcpy.Merge_management(wtdList, os.path.join(ingdb,'awc_huc12_wtds_merge'),'','ADD_SOURCE_INFO')
            arcpy.AddField_management(wtd_merge,'cat_ID_con','TEXT')
            arcpy.AddField_management(wtd_merge,'cat_ID','DOUBLE')
            arcpy.AddField_management(wtd_merge,'cat_ID_txt','TEXT')
            with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','cat_ID_con','cat_ID','cat_ID_txt']) as cur:
                for row in cur:
                    gridcode= int(row[0].split('_')[-1])
                    row[1] = roi + '_' + str(gridcode)
                    row[2] = int(gridcode)
                    row[3] = str(gridcode)
                    cur.updateRow(row)
                del(row)
            del(cur)
            arcpy.CopyFeatures_management(wtd_merge,os.path.join(outgdb, f'{roi}_AwcHuc12_wtds_merge' ))

            # Stop iteration timer
            reg_stop = time.time()
            reg_time = int (reg_stop - reg_start)
            print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=reg_time)})')
            print(f'{"*"*100}')

        except:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])
    else:
        print(f'{roi} not found - check inputs')
        sys.exit(f'{roi} not found - check inputs')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')




Cook_Inlet
['FDO_OBJECTID', 'FDO_Shape', 'NHDPlusID_index']
D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\cats_merge Indexed
1. Starting watershed for HUC 75004200000901 (660 remaining)
Elapsed time: (0:00:01)
************************************************************
2. Starting watershed for HUC 75004200001724 (659 remaining)
Elapsed time: (0:00:02)
************************************************************
3. Starting watershed for HUC 75004200001726 (658 remaining)
Elapsed time: (0:00:05)
************************************************************
4. Starting watershed for HUC 75004200001493 (657 remaining)
Elapsed time: (0:00:01)
************************************************************
5. Starting watershed for HUC 75004200004105 (656 remaining)
Elapsed time: (0:00:01)
************************************************************
6. Starting watershed for HUC 75004200009084 (655 remaining)
Elapsed time: (0:00:01)
************************************************************
7. Sta

## TROUBLESHOOTING BLOCK
### Zonal statistics as table is failing with unknown error when run on watershed_merge and slope/elev rasters if using 'ALL' statistics.
Try alternative methods.  Below is test chunk for iterating over a list of stats individually and join results back to a copy of the merged watershed table.
* Cannot Run ZonalStatistics because tool does not process overlapping polygons as individual features whereas ZonalStatistics as table will




In [12]:
### TEST CHUNK###

import os, arcpy,time, datetime
from arcpy.sa import *
arcpy.env.overwriteOutput = True
#
# testoutgdb = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\ArcGIS_Default\\ArcGIS_Default.gdb"
# wtd_merge = r"D:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\Cook_Inlet_AwcHuc12_wtds_merge"
# wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
# elev_rast = r"D:\\GIS\\AKSSF\\Cook_Inlet\\elev.tif"
# zstats = ['MIN_MAX_MEAN','STD']
# roi = 'Cook_Inlet'
#
# # Elevation variables
# wtd_merge_elev_table_name = roi + "_Watersheds_Merge_ElevZstats"
# wtd_merge_elev_table_path = os.path.join(testoutgdb, wtd_merge_elev_table_name)
#
# # list to store zonal stat tables
# wtdelevstats =[]
#
# # Create field mappings
# elev_fm = arcpy.FieldMap()
# elev_fms = arcpy.FieldMappings()
# for field in arcpy.ListFields(wtd_merge)[6:]:
#     elev_fm = arcpy.FieldMap()
#     elev_fm.addInputField(wtd_merge,field.name)
#     elev_fm.mergeRule = 'First'
#     # Set properties of the output name.
#     f_name = elev_fm.outputField
#     f_name.name = field.name
#     f_name.aliasName = field.name
#     elev_fm.outputField = f_name
#     elev_fms.addFieldMap(elev_fm)
#
# # Make copy of watershed merge input as table to join stats fields
# wtd_elev_metrics_table = arcpy.TableToTable_conversion(wtd_merge,
#                                                        testoutgdb,
#                                                        wtd_merge_elev_table_name,
#                                                        '',
#                                                        elev_fms,
#                                                        )
# # Add region identifier field for watershed tables                                                )
# arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
# # expression to calculate region field with roi name
# exp =  '"'+roi+'"'
# arcpy.CalculateField_management(wtd_elev_metrics_table,'region',exp)
#
# zstat_start = time.time()
# for stat in zstats:
#     outstattable = os.path.join(testoutgdb,f'{roi}_wtdElev{stat}')
#     zstat_start1 = time.time()
#     try:
#         print (f'running {stat}')
#         stat_table = ZonalStatisticsAsTable(in_zone_data = wtdmerge,
#                                             zone_field = wtd_cur_fields[0],
#                                             in_value_raster = elev_rast,
#                                             out_table = outstattable,
#                                             statistics_type=stat
#                                             )
#
#         stat_fields = [f.name for f in arcpy.ListFields(stat_table)]
#         arcpy.JoinField_management(wtd_elev_metrics_table,
#                                wtd_cur_fields[0],
#                                stat_table,
#                                wtd_cur_fields[0],
#                                stat_fields[5:]
#                                )
#
#         # Report time
#         zstat_stop1 = time.time()
#         zstat_time1 = int (zstat_stop1 - zstat_start1)
#         print(f'Watershed elev Zonal Stats for {stat} Elapsed time: ({datetime.timedelta(seconds=zstat_time1)})')
#         print(f'{"*"*100}')
#     except:
#         e = sys.exc_info()[1]
#         print(e.args[0])
#         arcpy.AddError(e.args[0])
# # Report time
# zstat_stop = time.time()
# zstat_time = int (zstat_stop - zstat_start)
# print(f'Watershed elev Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
# print(f'{"*"*100}')

## Section 4
Calculate Covariates


In [13]:
from arcpy.sa import *
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338) #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Lists for variables not needed at present time
#cat_asp_ztables = []
#wtd_asp_ztables = []
#cat_pernorth_taba_tables=[]

# Lists to store output tables
wtd_pernorth_taba_tables=[]
wtd_lp_tabint_tables = []
wtd_glac_tabint_tables = []
wtd_wet_taba_tables = []
cat_elev_ztables = []
wtd_elev_ztables = []
cat_slope_ztables = []
wtd_slope_ztables = []
lcld_Ztables = []

# Clear lists
cat_cur_fields = []
wtd_cur_fields = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Split data by type
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

#Limit to ci for testing
#regions = ['D:\\GIS\\AKSSF\\Cook_Inlet','D:\\GIS\\AKSSF\\Copper_River']
#Try with both data types
regions = ['D:\\GIS\\AKSSF\\Prince_William_Sound','D:\\GIS\\AKSSF\\Cook_Inlet','D:\\GIS\\AKSSF\\Copper_River']

for region in regions:
    roi = os.path.basename(region)
    # expression to calculate region field with roi name
    exp =  '"'+roi+'"'
    if roi in nhdplus_dat:
        lakes_fc = nhd_lakes_fc
        # Fields for update cursor
        cat_cur_fields = ['cat_ID_txt', 'NHDPlusID',"cat_ID_con"]
        wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
        print (f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
        print(f'{"*"*100}')
    # Set data and variables unique to regions with TauDEM Data
    elif roi in tauDem_dat:
        lakes_fc = tau_lakes_fc
        # Fields for update cursor
        if roi == 'Bristol_Bay':
            cat_cur_fields = ['cat_ID_txt', 'catID',"cat_ID_con"]
            wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
        else:
            cat_cur_fields = ['cat_ID_txt', 'gridcode',"cat_ID_con"]
            wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
        print (f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
        print(f'{"*"*100}')
    # Start iter timing function
    iteration_start = time.time()
    # Set workspace to region folder
    arcpy.env.workspace = region
    walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            # Set merged watersheds dataset
            if 'awc_huc12_wtds_merge'== filename:
                wtdpath = os.path.join(dirpath,filename)
                wtdname = roi +'_'+ filename
                # Make local copy projected in AKAlbers
                wtd_merge = os.path.join(dirpath, filename)
                print(f'Merged watershed dataset {filename} found')
                print(f'{"*"*100}')
                wtdfieldnames = []
                wtdlstFields = arcpy.ListFields(wtd_merge)
                for field in wtdlstFields:
                    wtdfieldnames.append(field.name)
                if str(wtd_cur_fields[0]) in wtdfieldnames:
                    print (f'{wtd_cur_fields[0]} field already in dataset')
                    print(f'{"*"*100}')
                else:
                    print (f'Adding {wtd_cur_fields[0]} field to watershed dataset {wtd_merge}')
                    print(f'{"*"*100}')
                    # add cat_ID_txt field and concat cat_ID + region
                    arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[0]),field_type='TEXT')
                    # populate cat_ID_txt
                    with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields[0:2]) as cur:
                        for row in cur:
                            strval = str(row[1])
                            row[0] = strval.replace('.0',"")
                            # Update rows
                            cur.updateRow(row)
                        del(row)
                    del(cur)
                if str(wtd_cur_fields[2]) in wtdfieldnames:
                    print (f'{wtd_cur_fields[2]} field already in dataset {wtd_merge}')
                    print(f'{"*"*100}')
                else:
                    print (f'Adding {wtd_cur_fields[2]} field to watershed dataset {wtd_merge}')
                    print(f'{"*"*100}')
                    # add cat_ID_con field and concat cat_ID + region
                    arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[2]),field_type='TEXT')
                    # populate cat_ID_txt
                    with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                        for row in cur:
                            strval = str(row[1])
                            row[2] = str(roi) +'_'+ strval.replace(".0","")
                            # Update rows
                            cur.updateRow(row)
                        del(row)
                    del(cur)

            # Select glaciers fc
            elif 'glaciers' == filename:
                # Make local copy projected in AKAlbers
                glacpath = os.path.join(dirpath, filename)
                glacname = roi+'_'+filename
                glac_fc = glacpath

            # Select elevation raster
            elif 'elev.tif' == filename:
                elev_rast = os.path.join(dirpath, filename)

            # # Select aspect raster
            # elif 'aspect' in filename:
            #     asp_rast = os.path.join(dirpath, filename)

            # Select north raster
            elif 'north.tif' == filename:
                nor_rast = os.path.join(dirpath, filename)

            # Select slope raster
            elif 'slope.tif' == filename:
                slope_rast = os.path.join(dirpath, filename)

            # Select wetland raster
            elif 'wetlands.tif' == filename:
                wet_rast = os.path.join(dirpath, filename)

            # Select catch_int fc (catchments of interest for region) and make a copy
            elif 'awc_huc12_catchment_outlets' == filename:
                # Make local copy projected in AKAlbers
                catspath = os.path.join(dirpath,filename)
                catsname = roi +"_"+filename
                cats = catspath
                catlstfields = arcpy.ListFields(cats)
                catfieldnames = []
                for field in catlstfields:
                    catfieldnames.append(field.name)
                if str(cat_cur_fields[0]) in catfieldnames:
                    print (f'{cat_cur_fields[0]} field already in dataset {cats}')
                    print(f'{"*"*100}')
                else:
                    print (f'Adding {cat_cur_fields[0]} field to catchment dataset {cats}')
                    print(f'{"*"*100}')
                    # add cat_ID_txt field
                    arcpy.AddField_management(cats, str(cat_cur_fields[0]), field_type='TEXT')
                    # populate cat_ID_txt
                    with arcpy.da.UpdateCursor(cats, cat_cur_fields[0:2]) as cur:
                        for row in cur:
                            strval = str(row[1])
                            row[0] = strval.replace('.0',"")
                            # Update rows
                            cur.updateRow(row)
                        del(row)
                    del(cur)
                if str(cat_cur_fields[2]) in catfieldnames:
                    print (f'{cat_cur_fields[2]} field already in dataset {cats}')
                    print(f'{"*"*100}')
                else:
                    print (f'Adding {cat_cur_fields[2]} field to catchment dataset {cats}')
                    print(f'{"*"*100}')
                    # add cat_ID_txt field & cat_ID + region concat field
                    arcpy.AddField_management(cats,str(cat_cur_fields[2]),field_type='TEXT')
                    # populate cat_ID_con
                    with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                        for row in cur:
                            strval = str(row[1])
                            row[2] = str(roi) +'_'+ strval.replace('.0',"")
                            # Update rows
                            cur.updateRow(row)
                        del(row)
                    del(cur)

    print (f'Calculating topographic metrics for catchments & watersheds of interest in {roi} region')
    print ('----------')
    print(f'Geodatabase: {outgdb}')
    print ('----------')
    print (f'Watershed Merge: {wtd_merge}')
    print (f'  Projection {arcpy.Describe(wtd_merge).spatialReference.name}')
    print ('----------')
    print (f'HUC12 Catchment Outlets: {cats}')
    print (f'  Projection {arcpy.Describe(cats).spatialReference.name}')
    print ('----------')
    print (f'Elevation Raster: {elev_rast}')
    print (f'  Projection: {arcpy.Describe(elev_rast).spatialReference.name}')
    print ('----------')
    print (f'North Aspect Raster: {nor_rast}')
    print (f'  Projection: {arcpy.Describe(nor_rast).spatialReference.name}')
    print ('----------')
    print (f'Wetlands Raster: {wet_rast}')
    print (f'  Projection {arcpy.Describe(wet_rast).spatialReference.name}')
    print ('----------')
    print (f'Slope Raster: {slope_rast}')
    print (f'  Projection {arcpy.Describe(slope_rast).spatialReference.name}')
    print ('----------')
    print (f'Lakes Ponds fc: {lakes_fc}')
    print (f'  Projection {arcpy.Describe(lakes_fc).spatialReference.name}')
    print ('----------')
    print (f'Glaciers fc: {glac_fc} ')
    print (f'  Projection {arcpy.Describe(glac_fc).spatialReference.name}')
    print ('----------')
    print (f'{arcpy.GetCount_management(wtd_merge)} Watersheds to process')
    print ('----------')
    print (f'Catchment intersect {cats} selected')
    print ('----------')

    # # Aspect variables
    # wtd_merge_asp_table_name = roi + "_AwcHuc12_wtd_mer_AspectZstats"
    # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
    # cat_asp_table_name = roi + "_AwcHuc12_cats_AspectZstats"
    # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)

    # Percent North variables
    wtd_merge_pernorth_table_name = roi + "_AwcHuc12_wtd_mer_PerNorth"
    wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
    # cat_pernorth_table_name = roi + "_AwcHuc12_cats_PercentNorth"
    # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

    # Elevation variables
    wtd_merge_elev_table_name = roi + "_AwcHuc12_wtd_mer_ElevZstats"
    wtd_merge_elev_table_path = os.path.join(outgdb, wtd_merge_elev_table_name)
    cat_elev_table_name = roi + "_AwcHuc12_cats_ElevZstats"
    cat_elev_table_path = os.path.join(outgdb, cat_elev_table_name)

    # Slope variables
    wtd_merge_slope_table_name = roi + "_AwcHuc12_wtd_mer_SlopeZstats"
    wtd_merge_slope_table_path = os.path.join(outgdb, wtd_merge_slope_table_name)
    cat_slope_table_name = roi + "_AwcHuc12_cats_SlopeZstats"
    cat_slope_table_path = os.path.join(outgdb, cat_slope_table_name)

    # Lakes Ponds variables
    wtd_merge_lp_table_name = roi + "_AwcHuc12_wtd_mer_PerLakes"
    wtd_merge_lp_table_path = os.path.join(outgdb, wtd_merge_lp_table_name)
    cat_lp_table_name = roi + "_AwcHuc12_cats_PerLakes"
    cat_lp_path = os.path.join(outgdb, cat_lp_table_name)

    # Wetlands variables
    wtd_merge_wetlands_table_name = roi + "_AwcHuc12_wtd_mer_PerWet"
    wtd_merge_wetlands_table_path = os.path.join(outgdb, wtd_merge_wetlands_table_name)
    cat_wetlands_table_name = roi + "AwcHuc12_cats_PerWet"
    cat_wetlands_table_path = os.path.join(outgdb, cat_wetlands_table_name)

    # Glaciers
    wtd_merge_glac_table_name = roi + "_AwcHuc12_wtd_mer_PerGlac"
    wtd_merge_glac_table_path = os.path.join(outgdb, wtd_merge_glac_table_name)
    cat_glac_table_name = roi + "_AwcHuc12_cats_Glaciers"
    cat_glac_table_path = os.path.join(outgdb, cat_glac_table_name)

    try: # Zonal Stats section
        print(f'Begin Slope zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
              f' region')
        # Statistics to run for watersheds - 'ALL' is not an option at this time as tool will fail with unknown error
        zstats = ['MIN_MAX_MEAN','STD']
        # Begin Zonal Stats
        zstat_start = time.time()
        zstat_start1 = time.time()

        # Watershed slope Zonal Statistics
        print(f'Calculating {roi} watershed slope zonal stats...')
        arcpy.env.snapRaster = slope_rast
        arcpy.env.cellSize = slope_rast

        # Create field mappings
        slope_fm = arcpy.FieldMap()
        slope_fms = arcpy.FieldMappings()
        for field in arcpy.ListFields(wtd_merge)[6:]:
            slope_fm = arcpy.FieldMap()
            slope_fm.addInputField(wtd_merge,field.name)
            slope_fm.mergeRule = 'First'
            # Set properties of the output name.
            f_name = slope_fm.outputField
            f_name.name = field.name
            f_name.aliasName = field.name
            slope_fm.outputField = f_name
            slope_fms.addFieldMap(slope_fm)

        # Make copy of watershed merge input as table to join stats fields
        wtd_slope_metrics_table = arcpy.TableToTable_conversion(wtd_merge,
                                                               outgdb,
                                                               wtd_merge_slope_table_name,
                                                               '',
                                                               slope_fms,
                                                               )
        # Add region identifier field for watershed tables                                                )
        arcpy.AddField_management(wtd_slope_metrics_table,'region',field_type='TEXT')
        arcpy.CalculateField_management(wtd_slope_metrics_table,'region',exp)

        for stat in zstats:
            outstattable = os.path.join(outgdb,f'{roi}_wtdSlope_{stat}')
            zstat_start1 = time.time()
            print (f'running {stat}')
            stat_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                zone_field = wtd_cur_fields[0],
                                                in_value_raster = slope_rast,
                                                out_table = outstattable,
                                                statistics_type=stat
                                                )

            stat_fields = [f.name for f in arcpy.ListFields(stat_table)]
            arcpy.JoinField_management(wtd_slope_metrics_table,
                                   wtd_cur_fields[0],
                                   stat_table,
                                   wtd_cur_fields[0],
                                   stat_fields[5:] # Keep only stat field/s
                                   )

            # Report time
            zstat_stop1 = time.time()
            zstat_time1 = int (zstat_stop1 - zstat_start1)
            print(f'Watershed Slope Zonal {stat} for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=zstat_time1)})')
            print(f'{"*"*100}')

        # Append watershed slope table to list
        wtd_slope_ztables.append(wtd_slope_metrics_table)


        # Elevation Zonal statistics  for watersheds
        print(f'Begin Elevation zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
              f' region')
        zstat_start2 = time.time()
        arcpy.env.snapRaster = elev_rast
        arcpy.env.cellSize = elev_rast

        # Create field mappings
        elev_fm = arcpy.FieldMap()
        elev_fms = arcpy.FieldMappings()
        for field in arcpy.ListFields(wtd_merge)[6:]:
            elev_fm = arcpy.FieldMap()
            elev_fm.addInputField(wtd_merge,field.name)
            elev_fm.mergeRule = 'First'
            # Set properties of the output name.
            f_name = elev_fm.outputField
            f_name.name = field.name
            f_name.aliasName = field.name
            elev_fm.outputField = f_name
            elev_fms.addFieldMap(elev_fm)

        # Make copy of watershed merge input as table to join stats fields
        wtd_elev_metrics_table = arcpy.TableToTable_conversion(wtd_merge,
                                                               outgdb,
                                                               wtd_merge_elev_table_name,
                                                               '',
                                                               elev_fms,
                                                               )
        # Add region identifier field for watershed tables                                                )
        arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
        arcpy.CalculateField_management(wtd_elev_metrics_table,'region',exp)

        for stat in zstats:
            outstattable = os.path.join(outgdb,f'{roi}_wtdElev_{stat}')
            zstat_start1 = time.time()
            print (f'running {stat}')
            stat_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                zone_field = wtd_cur_fields[0],
                                                in_value_raster = elev_rast,
                                                out_table = outstattable,
                                                statistics_type=stat
                                                )

            stat_fields = [f.name for f in arcpy.ListFields(stat_table)]
            arcpy.JoinField_management(wtd_elev_metrics_table,
                                   wtd_cur_fields[0],
                                   stat_table,
                                   wtd_cur_fields[0],
                                   stat_fields[5:] # Keep only stat field/s
                                   )

            # Report time
            zstat_stop2 = time.time()
            zstat_time2 = int (zstat_stop2 - zstat_start2)
            print(f'Watershed Elevation Zonal {stat} for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=zstat_time2)})')
            print(f'{"*"*100}')
        # Append watershed elev table to list
        wtd_elev_ztables.append(wtd_elev_metrics_table)


        # Elevation zonal statistics for catchments
        print(f'Calculating {roi} catchment elevation zonal stats...')
        zstat_start3 = time.time()
        arcpy.env.snapRaster = elev_rast
        arcpy.env.cellSize = elev_rast
        cat_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                        zone_field = cat_cur_fields[0],
                                                        in_value_raster = elev_rast,
                                                        out_table = cat_elev_table_path,
                                                        statistics_type='ALL'
                                                        )
        # Add region identifier field for catchment table
        arcpy.AddField_management(cat_elev_metrics_table,'region',field_type='TEXT')
        # Add cat_ID_Con field
        arcpy.AddField_management(cat_elev_metrics_table,'cat_ID_con',field_type='TEXT')

        # Update fields
        with arcpy.da.UpdateCursor(cat_elev_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
            for row in cur:
                row[0] = roi
                strval = str(row[1])
                row[2] = roi+"_"+strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Append catchment elev table to list
        cat_elev_ztables.append(cat_elev_metrics_table)
        # Report time
        zstat_stop3 = time.time()
        zstat_time3 = int (zstat_stop3 - zstat_start3)
        print(f'Elevation Zonal Stats for {roi} catchments complete.\nElapsed time: ({datetime.timedelta(seconds=zstat_time3)})')
        print(f'{"*"*100}')

        # Slope zonal statistics for catchments
        zstat_start4 = time.time()
        print(f'Calculating {roi} catchment slope zonal stats...')
        arcpy.env.snapRaster = slope_rast
        arcpy.env.cellSize = slope_rast
        cat_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                        zone_field = cat_cur_fields[0],
                                                        in_value_raster = slope_rast,
                                                        out_table = cat_slope_table_path,
                                                        statistics_type='ALL'
                                                        )
        # Add region identifier field for catchment table
        arcpy.AddField_management(cat_slope_metrics_table,'region',field_type='TEXT')
        # Add cat_ID_Con field
        arcpy.AddField_management(cat_slope_metrics_table,'cat_ID_con',field_type='TEXT')

        # Update region field
        with arcpy.da.UpdateCursor(cat_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
            for row in cur:
                row[0] = roi
                strval =str(row[1])
                row[2] = roi+"_"+strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Append catchment slope table to list
        cat_slope_ztables.append(cat_slope_metrics_table)
        # Report time
        zstat_stop4 = time.time()
        zstat_time4 = int (zstat_stop4 - zstat_start4)
        print(f'Slope Zonal Stats for {roi} catchments complete.\nElapsed time: ({datetime.timedelta(seconds=zstat_time4)})')
        print(f'{"*"*100}')


        # # Aspect Zonal statistics  for watersheds
        # print(f'Calculating {roi} watershed aspect zonal stats...')
        # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="cat_ID_txt",
        #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
        #                                                statistics_type='ALL')
        # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
        # Add cat_ID_Con field
        # arcpy.AddField_management(wtd_asp_metrics_table,'cat_ID_con',field_type='TEXT')
        # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
        # Update region field
        # with arcpy.da.UpdateCursor(wtd_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
        #     for row in cur:
        #         row[0] = roi
        #         strval = str(row[1])
        #         row[2] = roi+"_"+strval.replace(".0","")
        #         # Update
        #         cur.updateRow(row)
        #     del(row)
        # del(cur)
        # wtd_asp_ztables.append(wtd_asp_metrics_table)

        # # Aspect Zonal statistics for catchments
        # print(f'Calculating {roi} catchment aspect zonal stats...')
        # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="cat_ID_txt",
        #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
        #                                                statistics_type='ALL')
        # arcpy.AddField_management(cat_asp_metrics_table, 'region', field_type='TEXT')
        # Add cat_ID_Con field
        # arcpy.AddField_management(cat_asp_metrics_table,'cat_ID_con',field_type='TEXT')
        # arcpy.CalculateField_management(cat_asp_metrics_table, 'region', 'roi')
        # Update region field
        # with arcpy.da.UpdateCursor(cat_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
        #     for row in cur:
        #         strval = str(row[1])
        #         row[2] = roi+"_"+strval.replace(".0","")
        #         # Update
        #         cur.updateRow(row)
        #     del(row)
        # del(cur)
        # cat_asp_ztables.append(cat_asp_metrics_table)

        zstat_stop = time.time()
        zstat_time = int (zstat_stop - zstat_start)
        print(f'All Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
        print(f'{"*"*100}')

        # Tabulate Area with north grid and watersheds
        tabarea_start = time.time()
        tabarea_start1 = time.time()
        print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
        print(f'{"*"*100}')
        # Percent North Tabulate area for watersheds
        wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                      zone_field= wtd_cur_fields[0],
                                                      in_class_data=nor_rast,
                                                      class_field="Value",
                                                      out_table = wtd_merge_pernorth_table_path
                                                      )
        # Add region and percent north fields
        arcpy.AlterField_management(wtd_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
        arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
        arcpy.AddField_management(wtd_per_north_tabarea, 'AwcHuc12_wtd_north_per', field_type='Float')
        arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[0], field_type='TEXT')
        arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[2], field_type='TEXT')
        wtdnorfields = [f.name for f in arcpy.ListFields(wtd_per_north_tabarea)]
        #print (wtdnorfields)
        with arcpy.da.UpdateCursor(wtd_per_north_tabarea, wtdnorfields) as cur:
            for row in cur:
                strval = str(row[1])
                row[4] = roi
                row[5] = row[3]/(row[3]+row[2])*100
                row[6] = strval.replace('.0','')
                row[7] = roi +'_'+ strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Drop UPPERCASE field form tab area
        arcpy.DeleteField_management(wtd_per_north_tabarea,'CAT_ID_TXT_DEL')
        # Append watershed percent north table to list
        wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)
        # Report tab area times
        tabarea_stop1 = time.time()
        tabarea_time1 = int (tabarea_stop1 - tabarea_start1)
        print(f'Watershed percent north Tabulate area/intersections for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=tabarea_time1)})')
        print(f'{"*"*100}')

        # Percent Lakes Ponds using Tabulate Intersection for watersheds
        print(f'Begin watershed percent lakes ponds for {roi}')
        tabarea_start2 = time.time()
        wtd_lp_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                            zone_fields=wtd_cur_fields[0],
                                                            in_class_features=lakes_fc,
                                                            out_table=wtd_merge_lp_table_path,
                                                            class_fields='Ftype',
                                                            out_units="SQUARE_METERS"
                                                            )
        # Add region and cat id fields
        arcpy.AlterField_management(wtd_lp_tabint,'PERCENTAGE','AwcHuc12_wtd_lake_per','AwcHuc12_wtd_lake_per')
        arcpy.AlterField_management(wtd_lp_tabint,'AREA','AwcHuc12_wtd_lake_area_sqm','AwcHuc12_wtd_lake_area_sqm')
        arcpy.AddField_management(wtd_lp_tabint, 'region', field_type='TEXT')
        arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[1], field_type='TEXT')
        arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[2], field_type='TEXT')
        wtdlpfields = [f.name for f in arcpy.ListFields(wtd_lp_tabint)]
        #print (wtdlpfields)
        with arcpy.da.UpdateCursor(wtd_lp_tabint, wtdlpfields) as cur:
            for row in cur:
                strval = str(row[1])
                row[5] = roi
                row[6] = strval.replace('.0','')
                row[7] = roi +'_'+ strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)

        # Append watershed lakes ponds table to list
        wtd_lp_tabint_tables.append(wtd_lp_tabint)
        # Report tab area times
        tabarea_stop2 = time.time()
        tabarea_time2 = int (tabarea_stop2 - tabarea_start2)
        print(f'Percent Lakes Tabulate area/intersections for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=tabarea_time2)})')
        print(f'{"*"*100}')

        # Percent glaciers using Tabulate Intersection for watersheds
        tabarea_start3 = time.time()
        print(f'Begin tabulate intersection between {glac_fc} and watersheds in {roi} region')
        print(f'{"*"*100}')
        wtd_glac_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                            zone_fields=wtd_cur_fields[0],
                                                            in_class_features=glac_fc,
                                                            out_table=wtd_merge_glac_table_path,
                                                            class_fields='O1Region',
                                                            out_units="SQUARE_METERS"
                                                            )
        # Add region and cat id fields
        arcpy.AlterField_management(wtd_glac_tabint,'PERCENTAGE','AwcHuc12_wtd_glacier_per','AwcHuc12_wtd_glacier_per')
        arcpy.AlterField_management(wtd_glac_tabint,'AREA','AwcHuc12_wtd_glacier_area_sqm','AwcHuc12_wtd_glacier_area_sqm')
        arcpy.AddField_management(wtd_glac_tabint, 'region', field_type='TEXT')
        arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[1], field_type='TEXT')
        arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[2], field_type='TEXT')
        wtdglacfields = [f.name for f in arcpy.ListFields(wtd_glac_tabint)]
        #print (wtdglacfields)
        with arcpy.da.UpdateCursor(wtd_glac_tabint, wtdglacfields) as cur:
            for row in cur:
                strval = str(row[1])
                row[5] = roi
                row[6] = strval.replace('.0','')
                row[7] = roi +'_'+ strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Append watershed percent glacier table to list
        wtd_glac_tabint_tables.append(wtd_glac_tabint)
        # Report tab area times
        tabarea_stop3 = time.time()
        tabarea_time3 = int (tabarea_stop3 - tabarea_start3)
        print(f'Percent Glacier Tabulate area/intersections for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=tabarea_time3)})')
        print(f'{"*"*100}')

        # Tabulate Area with wetlands grid and watersheds
        tabarea_start4 = time.time()
        print(f'Begin tabulate intersection between {wet_rast} and watersheds in {roi} region')
        print(f'{"*"*100}')
        # Wetlands tabulate area for watersheds
        wtd_per_wet_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                      zone_field= wtd_cur_fields[0],
                                                      in_class_data=wet_rast,
                                                      class_field="Value",
                                                      out_table=wtd_merge_wetlands_table_path
                                                      )
        # Add region and percent wet fields
        arcpy.AlterField_management(wtd_per_wet_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
        arcpy.AddField_management(wtd_per_wet_tabarea, 'region', field_type='TEXT')
        arcpy.AddField_management(wtd_per_wet_tabarea, 'AwcHuc12_wtd_wet_per', field_type='Float')
        arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[0], field_type='TEXT')
        arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[2], field_type='TEXT')
        wtdwetfields = [f.name for f in arcpy.ListFields(wtd_per_wet_tabarea)]
        #print (wtdwetfields)
        with arcpy.da.UpdateCursor(wtd_per_wet_tabarea, wtdwetfields) as cur:
            for row in cur:
                strval = str(row[1])
                row[4] = roi
                row[5] = row[3]/(row[3]+row[2])*100
                row[6] = strval.replace('.0','')
                row[7] = roi +'_'+ strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Drop UPPERCASE field form tab area
        arcpy.DeleteField_management(wtd_per_wet_tabarea,'CAT_ID_TXT_DEL')
        # Append watershed percent wetlands table to list
        wtd_wet_taba_tables.append(wtd_per_wet_tabarea)
        # Report tab area times
        tabarea_stop4 = time.time()
        tabarea_time4 = int (tabarea_stop4 - tabarea_start4)
        print(f'Percent Wetlands Tabulate area/intersections for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=tabarea_time4)})')
        print(f'{"*"*100}')

        # # Percent North Tabulate Area for catchments
        # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='cat_ID_con',
        #                                             in_class_data=nor_rast,"Value",
        #                                             out_table=cat_pernorth_table_path)

        # # Add and calculate region identifier field for catchment table
        # arcpy.AlterField_management(cat_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
        # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
        # arcpy.AddField_management(cat_per_north_tabarea, 'cat_north_per', field_type='Float')
        # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[0], field_type='TEXT')
        # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[2], field_type='TEXT')
        # catnorfields = [f.name for f in arcpy.ListFields(cat_per_north_tabarea)]
        # print (catnorfields)
        # with arcpy.da.UpdateCursor(cat_per_north_tabarea,catnorfields) as cur:
        #     for row in cur:
        #         strval = str(row[1])
        #         row[4] = roi
        #         row[5] = row[3]/(row[3]+row[2])*100
        #         row[6] = strval.replace('.0','')
        #         row[7] = roi +'_'+ strval.replace(".0","")
        #         # Update
        #         cur.updateRow(row)
        #     del(row)
        # del(cur)
        # Drop UPPERCASE field form tab area
        # arcpy.DeleteField_management(cat_per_north_tabarea,'CAT_ID_TXT_DEL')
        # # Append catchment percent north table to list
        # cat_pernorth_taba_tables.append(cat_per_north_tabarea)
        # Report tab area times
        tabarea_stop = time.time()
        tabarea_time = int (tabarea_stop - tabarea_start)
        print(f'Tabulate area/intersections for {roi} complete\nElapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
        print(f'{"*"*100}')

        # Begin LCLD calculations
        walk = arcpy.da.Walk(lcld_folder, datatype='RasterDataset')
        for dirpath, dirnames, filenames in walk:
            for filename in filenames:
                raspath = os.path.join(dirpath, filename)
                year = filename[0:4]
                lcld_outname = roi+'_AwcHuc12_lcld_'+str(year)+'_zStats'
                lcld_outpath = os.path.join(outgdb, lcld_outname)
                print(f'Year: {year} - raster path {raspath}')
                colname = 'AwcHuc12_wtd_lcld_mn_' + str(year)
                # lcld zonal statistics as table for all akssf watersheds
                print(f'Calculating {filename} zonal stats for all {roi} watersheds...')
                #arcpy.env.snapRaster = raspath
                #arcpy.env.cellSize = raspath

                # Begin Zonal Stats
                lcldzstat_start = time.time()
                print(f'Begin zonal stats for {filename}')
                lcld_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = 'cat_ID_con',
                                                                in_value_raster = raspath,
                                                                out_table = lcld_outpath,
                                                                statistics_type='MEAN'
                                                                )
                # Append zTable to table list
                lcld_Ztables.append(lcld_outpath)
                arcpy.AlterField_management(lcld_table,'MEAN', colname,colname)
                proc_list = [row[0] for row in arcpy.da.SearchCursor(lcld_table,'cat_ID_con')]
                lcldzstat_stop = time.time()
                lcldzstat_time = int (lcldzstat_stop - lcldzstat_start)
                print(f'Zonal Stats for {filename} - Elapsed time: ({datetime.timedelta(seconds=lcldzstat_time)})')


    except:
        e = sys.exc_info()[1]
        print(f'ERRFLAG!!! = {e.args[0]}')
        arcpy.AddError(e.args[0])

    iter_stop = time.time()
    iter_time = int(iter_stop - iteration_start)
    print(f'All Covariates for {roi} completed.\nElapsed time: ({datetime.timedelta(seconds=iter_time)})')
    print(f'{"*"*100}')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')



Prince_William_Sound in ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound'] TauDEM list, using cat_fields ['cat_ID_txt', 'gridcode', 'cat_ID_con'] and watershed fields ['cat_ID_txt', 'cat_ID', 'cat_ID_con']
****************************************************************************************************
Adding cat_ID_txt field to catchment dataset D:\GIS\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb\awc_huc12_catchment_outlets
****************************************************************************************************
Adding cat_ID_con field to catchment dataset D:\GIS\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb\awc_huc12_catchment_outlets
****************************************************************************************************
Merged watershed dataset awc_huc12_wtds_merge found
****************************************************************************************************
cat_ID_txt field already in dataset
***************************************

Examine LCLD tables and merge/export
* Discovered one watershed for PWS (Prince_William_Sound_23854) only has 6 years of LCLD data attributed to it.  Modis Coverage along the Coastline and PWS in particular is poor and many of the Islands/Coastlines have limited coverage.

In [14]:
arcpy.env.workspace = outgdb
lcld_Ztables = [t for t in arcpy.ListTables('*_lcld_*')]
from collections import OrderedDict
lcld_Dict = {}
dfs = []
lcld_cols = []
drop_cols = ['OBJECTID','ZONE_CODE', 'AREA', 'COUNT']
for table in lcld_Ztables:
    cols = [f.name for f in arcpy.ListFields(table)]
    search_cols = [x for x in cols if x not in drop_cols]
    #cols.remove(drop_cols)
    print(search_cols)
    lcld_cols.append(search_cols)
    tblname = table[-16:]
    print(tblname)
    with arcpy.da.SearchCursor(table, search_cols) as cur:
        for row in cur:

            append_value(lcld_Dict,row[0],row[1])

lcld_cols = [item for sublist in lcld_cols for item in sublist]
lcld_cols = list(OrderedDict.fromkeys(lcld_cols))
print(lcld_cols)

#Create list of tuples containing unique id followed by mean lcld for years 2001-2019
row_values = [(k,)+tuple(v) for k,v in lcld_Dict.items()]

['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2001']
lcld_2001_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2002']
lcld_2002_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2003']
lcld_2003_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2004']
lcld_2004_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2005']
lcld_2005_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2006']
lcld_2006_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2007']
lcld_2007_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2008']
lcld_2008_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2009']
lcld_2009_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2010']
lcld_2010_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2011']
lcld_2011_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2012']
lcld_2012_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2013']
lcld_2013_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2014']
lcld_2014_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2015']
lcld_2015_zStats
['cat_ID_con', 'AwcHuc12_wtd_lcld_mn_2016']
lcld_2016_zStats
['cat_ID_con', 'AwcHuc12

Create an empty table to append yearly data
 * Attempting to convert to df and numpy arrays has been unsuccessful

In [15]:
lcld_table = arcpy.CreateTable_management(outgdb,'AKSSF_ALL_LCLD_mn')
arcpy.AddField_management(lcld_table.getOutput(0),lcld_cols[0],'TEXT')
cur = arcpy.da.InsertCursor(lcld_table.getOutput(0),lcld_cols[0])
# Populate table with cat_ID_con values stored in list of lcld tuples
try:
    for row in row_values:
        cur.insertRow([row[0]])
        del(row)
    del(cur)
except:
    e = sys.exc_info()[1]
    print(f'ERRFLAG!!! = {e.args[0]}\n')
    # Get the traceback object
    tb = sys.exc_info()[2]
    tbinfo = traceback.format_tb(tb)[0]
    # Concatenate information together concerning the error into a message string
    pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
    msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
    # Return Python error messages for use in script tool or Python window
    arcpy.AddError(pymsg)
    arcpy.AddError(msgs)
    # Print Python error messages for use in Python / Python window
    print(pymsg)
    print(msgs)
    arcpy.AddError(e.args[0])
print(f'LCDL table has {arcpy.GetCount_management(lcld_table)} records of {len(row_values)} input rows')

LCDL table has 1018 records of 1018 input rows


In [16]:

years = [x for x in range(2001,2020)]
lcldyr_Dict = {}
for y in years:
    for table in lcld_Ztables:
        if str(y) in table:
            append_value(lcldyr_Dict,str(y),table)
#print(lcldyr_Dict)

for k, v in lcldyr_Dict.items():
    year = k
    intables = v
    print (f'{year} using tables {intables} with val {v}\n')
    tempTable = arcpy.Merge_management(v,f'memory\\tempTable')
    infields = [f.name for f in arcpy.ListFields(tempTable)]
    print(f'Joining field {infields[5]} from {tempTable} to {lcld_table}')
    print(f'{"*"*100}\n')
    arcpy.JoinField_management(lcld_table,lcld_cols[0],tempTable,lcld_cols[0],infields[5])

2001 using tables ['Prince_William_Sound_AwcHuc12_lcld_2001_zStats', 'Cook_Inlet_AwcHuc12_lcld_2001_zStats', 'Copper_River_AwcHuc12_lcld_2001_zStats'] with val ['Prince_William_Sound_AwcHuc12_lcld_2001_zStats', 'Cook_Inlet_AwcHuc12_lcld_2001_zStats', 'Copper_River_AwcHuc12_lcld_2001_zStats']

Joining field AwcHuc12_wtd_lcld_mn_2001 from memory\tempTable to D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\AKSSF_ALL_LCLD_mn
****************************************************************************************************

2002 using tables ['Prince_William_Sound_AwcHuc12_lcld_2002_zStats', 'Cook_Inlet_AwcHuc12_lcld_2002_zStats', 'Copper_River_AwcHuc12_lcld_2002_zStats'] with val ['Prince_William_Sound_AwcHuc12_lcld_2002_zStats', 'Cook_Inlet_AwcHuc12_lcld_2002_zStats', 'Copper_River_AwcHuc12_lcld_2002_zStats']

Joining field AwcHuc12_wtd_lcld_mn_2002 from memory\tempTable to D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\AKSSF_ALL_LCLD_mn
**************************************************

In [17]:
# Convert to df and examine
# Make catchment lcld df
lcld_df = pd.DataFrame()
lcld_field_list = []
for field in arcpy.ListFields(lcld_table):
    lcld_field_list.append(field.name)
lcld_arr = arcpy.da.TableToNumPyArray(lcld_table,lcld_field_list)
lcld_df = pd.DataFrame(lcld_arr)
#lcld_df = lcld_df.drop(["OBJECTID","AwcHuc12_cat_elev_ZONE_CODE"],axis=1)
lcld_df = lcld_df.set_index('cat_ID_con')
#dfs.append(lcld_df)
lcld_df


,OBJECTID,AwcHuc12_wtd_lcld_mn_2001,AwcHuc12_wtd_lcld_mn_2002,AwcHuc12_wtd_lcld_mn_2003,AwcHuc12_wtd_lcld_mn_2004,AwcHuc12_wtd_lcld_mn_2005,AwcHuc12_wtd_lcld_mn_2006,AwcHuc12_wtd_lcld_mn_2007,AwcHuc12_wtd_lcld_mn_2008,AwcHuc12_wtd_lcld_mn_2009,AwcHuc12_wtd_lcld_mn_2010,AwcHuc12_wtd_lcld_mn_2011,AwcHuc12_wtd_lcld_mn_2012,AwcHuc12_wtd_lcld_mn_2013,AwcHuc12_wtd_lcld_mn_2014,AwcHuc12_wtd_lcld_mn_2015,AwcHuc12_wtd_lcld_mn_2016,AwcHuc12_wtd_lcld_mn_2017,AwcHuc12_wtd_lcld_mn_2018,AwcHuc12_wtd_lcld_mn_2019
cat_ID_con,,,,,,,,,,,,,,,,,,,,
Prince_William_Sound_3038,1,508.555029,508.128684,463.393467,500.807675,476.085032,508.915857,508.936782,524.235347,520.363026,504.248257,497.990594,519.367798,516.442898,487.399409,459.335113,438.086274,491.189930,501.663282,458.886742
Prince_William_Sound_13227,2,514.252371,505.159319,480.000000,516.000000,487.000000,511.800851,511.383761,513.995746,520.000000,506.243233,509.960170,533.761740,524.000000,488.000000,474.740458,469.880510,485.916087,492.000000,486.102938
Prince_William_Sound_17027,3,515.999191,517.335258,471.393950,509.214663,492.210631,511.377111,519.209941,531.016164,522.439286,510.353149,500.020681,530.855632,524.138335,492.491116,453.116534,441.996546,504.222195,503.255956,485.734227
Prince_William_Sound_17697,4,522.877156,531.425302,496.266759,521.571098,501.987227,520.330224,533.479922,554.323597,545.339405,522.740585,524.488093,548.313281,533.971525,502.494092,482.508580,475.545204,515.761395,506.677339,502.772358
Prince_William_Sound_18357,5,518.928843,519.123082,479.084527,511.788717,493.635879,521.300112,529.690076,542.799450,532.419252,516.344181,510.726020,538.696188,528.862207,496.909217,461.282085,462.003602,506.690720,496.491091,477.655689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Copper_River_75003900029086,1014,428.084574,486.099145,434.007325,470.238494,426.929563,461.685259,479.305537,481.603447,482.134102,472.782352,458.794501,485.641902,487.820103,423.330805,414.759543,410.158446,470.742159,451.558074,439.999561
Copper_River_75003900044552,1015,506.135406,501.755675,441.515220,487.940728,475.953439,497.688529,502.735879,514.357314,500.875332,505.846055,494.667803,529.897337,514.894985,479.218491,414.404403,425.842085,496.148778,491.614218,470.632806
Copper_River_75003900054944,1016,471.212499,494.243152,422.678038,464.086918,439.834718,477.171759,486.436765,488.877583,489.949104,475.444300,476.375077,505.542194,493.379799,470.743883,423.433889,413.119983,478.510037,472.765282,434.095321


In [18]:
# Export LCLD table
arcpy.TableToTable_conversion(lcld_table,outdir,'AKSSF_AWC_HUC12_wtd_lcld_mn.csv')
print(f'Table exported...')


Table exported...


## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [19]:

# Input table names/paths
cat_elev_ztables = [t for t in arcpy.ListTables('*_AwcHuc12_cats_ElevZstats')]
cat_slope_ztables = [t for t in arcpy.ListTables('*_AwcHuc12_cats_SlopeZstats')]
wtd_elev_ztables = [t for t in arcpy.ListTables('*_AwcHuc12_wtd_mer_ElevZstats')]
wtd_slope_ztables = [t for t in arcpy.ListTables('*_AwcHuc12_wtd_mer_SlopeZstats')]
wtd_pernorth_taba_tables = [t for t in arcpy.ListTables('*_AwcHuc12_wtd_mer_PerNorth')]
wtd_wet_taba_tables = [t for t in arcpy.ListTables('*_AwcHuc12_wtd_mer_PerWet')]
wtd_glac_tabint_tables = [t for t in arcpy.ListTables('*_AwcHuc12_wtd_mer_PerGlac')]
wtd_lp_tabint_tables = [t for t in arcpy.ListTables('*_AwcHuc12_wtd_mer_PerLakes')]

# Output Table names/paths
wtd_per_north_table_out = os.path.join(outgdb, 'AKSSF_awchuc12_wtd_north_per')
cat_elev_table_out = os.path.join(outgdb,'AKSSF_awchuc12_cat_elev')
cat_slope_table_out = os.path.join(outgdb,'AKSSF_awchuc12_cat_slope')
wtd_elev_table_out = os.path.join(outgdb, 'AKSSF_awchuc12_wtd_elev')
wtd_per_glac_table_out = os.path.join(outgdb, 'AKSSF_awchuc12_wtd_glacier_per')
wtd_per_lp_table_out = os.path.join(outgdb, 'AKSSF_awchuc12_wtd_lakepond_per')
wtd_slope_table_out = os.path.join(outgdb, 'AKSSF_awchuc12_wtd_slope')
wtd_wet_table_out = os.path.join(outgdb, 'AKSSF_awchuc12_wtd_wetland_per')

# Merge all regional tables together
outtables = []
wtd_per_north = arcpy.Merge_management(wtd_pernorth_taba_tables, wtd_per_north_table_out)
arcpy.AlterField_management(wtd_per_north,"VALUE_0","AwcHuc12_non_north_area","AwcHuc12_non_north_area")
arcpy.AlterField_management(wtd_per_north,"VALUE_1","AwcHuc12_north_area","AwcHuc12_north_area")
outtables.append(wtd_per_north)
cat_elev = arcpy.Merge_management(cat_elev_ztables, cat_elev_table_out)
outtables.append(cat_elev)
wtd_elev = arcpy.Merge_management(wtd_elev_ztables, wtd_elev_table_out)
outtables.append(wtd_elev)
wtd_slope = arcpy.Merge_management(wtd_slope_ztables, wtd_slope_table_out)
outtables.append(wtd_slope)
cat_slope = arcpy.Merge_management(cat_slope_ztables, cat_slope_table_out)
outtables.append(cat_slope)
wtd_wet = arcpy.Merge_management(wtd_wet_taba_tables, wtd_wet_table_out)
arcpy.AlterField_management(wtd_wet,"VALUE_0","AwcHuc12_non_wetland_area","AwcHuc12_non_wetland_area")
arcpy.AlterField_management(wtd_wet,"VALUE_1","AwcHuc12_wetland_area","AwcHuc12_wetland_area")
outtables.append(wtd_wet)
wtd_glac = arcpy.Merge_management(wtd_glac_tabint_tables, wtd_per_glac_table_out)
outtables.append(wtd_glac)
wtd_lp = arcpy.Merge_management(wtd_lp_tabint_tables, wtd_per_lp_table_out)
outtables.append(wtd_lp)
print ('Tables merged')
print('----------')


Tables merged
----------


In [20]:
#Set up field dictionary
elevDict = { 'ZONE_CODE': ('AwcHuc12_cat_elev_ZONE_CODE', 'AwcHuc12_wtd_elev_ZONE_CODE'),
         'COUNT': ('AwcHuc12_cat_elev_COUNT', 'AwcHuc12_wtd_elev_COUNT'),
          'AREA': ('AwcHuc12_cat_elev_AREA', 'AwcHuc12_wtd_elev_AREA'),
          'MIN': ('AwcHuc12_cat_elev_MIN', 'AwcHuc12_wtd_elev_MIN'),
          'MAX': ('AwcHuc12_cat_elev_MAX', 'AwcHuc12_wtd_elev_MAX'),
          'RANGE': ('AwcHuc12_cat_elev_RANGE', 'AwcHuc12_wtd_elev_RANGE'),
          'MEAN': ('AwcHuc12_cat_elev_MEAN', 'AwcHuc12_wtd_elev_MEAN'),
          'STD': ('AwcHuc12_cat_elev_STD', 'AwcHuc12_wtd_elev_STD'),
          'SUM': ('AwcHuc12_cat_elev_SUM', 'AwcHuc12_wtd_elev_SUM'),
          'VARIETY': ('AwcHuc12_cat_elev_VARIETY', 'AwcHuc12_wtd_elev_VARIETY'),
          'MAJORITY': ('AwcHuc12_cat_elev_MAJORITY', 'AwcHuc12_wtd_elev_MAJORITY'),
          'MINORITY': ('AwcHuc12_cat_elev_MINORITY', 'AwcHuc12_wtd_elev_MINORITY'),
          'MEDIAN': ('AwcHuc12_cat_elev_MEDIAN', 'AwcHuc12_wtd_elev_MEDIAN'),
          'PCT90': ('AwcHuc12_cat_elev_PCT90', 'AwcHuc12_wtd_elev_PCT90')
         }

slopeDict = { 'ZONE_CODE': ('AwcHuc12_cat_slope_ZONE_CODE', 'AwcHuc12_wtd_slope_ZONE_CODE'),
         'COUNT': ('AwcHuc12_cat_slope_COUNT', 'AwcHuc12_wtd_slope_COUNT'),
          'AREA': ('AwcHuc12_cat_slope_AREA', 'AwcHuc12_wtd_slope_AREA'),
          'MIN': ('AwcHuc12_cat_slope_MIN', 'AwcHuc12_wtd_slope_MIN'),
          'MAX': ('AwcHuc12_cat_slope_MAX', 'AwcHuc12_wtd_slope_MAX'),
          'RANGE': ('AwcHuc12_cat_slope_RANGE', 'AwcHuc12_wtd_slope_RANGE'),
          'MEAN': ('AwcHuc12_cat_slope_MEAN', 'AwcHuc12_wtd_slope_MEAN'),
          'STD': ('AwcHuc12_cat_slope_STD', 'AwcHuc12_wtd_slope_STD'),
          'SUM': ('AwcHuc12_cat_slope_SUM', 'AwcHuc12_wtd_slope_SUM'),
          'VARIETY': ('AwcHuc12_cat_slope_VARIETY', 'AwcHuc12_wtd_slope_VARIETY'),
          'MAJORITY': ('AwcHuc12_cat_slope_MAJORITY', 'AwcHuc12_wtd_slope_MAJORITY'),
          'MINORITY': ('AwcHuc12_cat_slope_MINORITY', 'AwcHuc12_wtd_slope_MINORITY'),
          'MEDIAN': ('AwcHuc12_cat_slope_MEDIAN', 'AwcHuc12_wtd_slope_MEDIAN'),
          'PCT90': ('AwcHuc12_cat_slope_PCT90', 'AwcHuc12_wtd_slope_PCT90')
         }

# Rename fields for elevation tables
for field in arcpy.ListFields(wtd_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][1]
        newalias = elevDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_elev, keyval, newname, newalias)

for field in arcpy.ListFields(cat_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][0]
        newalias = elevDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_elev, keyval, newname, newalias)

# Rename fields for slope tables
for field in arcpy.ListFields(wtd_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][1]
        newalias = slopeDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_slope, keyval, newname, newalias)

for field in arcpy.ListFields(cat_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][0]
        newalias = slopeDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_slope, keyval, newname, newalias)

MIN AwcHuc12_wtd_elev_MIN
MAX AwcHuc12_wtd_elev_MAX
MEAN AwcHuc12_wtd_elev_MEAN
STD AwcHuc12_wtd_elev_STD
ZONE_CODE AwcHuc12_cat_elev_ZONE_CODE
COUNT AwcHuc12_cat_elev_COUNT
AREA AwcHuc12_cat_elev_AREA
MIN AwcHuc12_cat_elev_MIN
MAX AwcHuc12_cat_elev_MAX
RANGE AwcHuc12_cat_elev_RANGE
MEAN AwcHuc12_cat_elev_MEAN
STD AwcHuc12_cat_elev_STD
SUM AwcHuc12_cat_elev_SUM
VARIETY AwcHuc12_cat_elev_VARIETY
MAJORITY AwcHuc12_cat_elev_MAJORITY
MINORITY AwcHuc12_cat_elev_MINORITY
MEDIAN AwcHuc12_cat_elev_MEDIAN
PCT90 AwcHuc12_cat_elev_PCT90
MIN AwcHuc12_wtd_slope_MIN
MAX AwcHuc12_wtd_slope_MAX
MEAN AwcHuc12_wtd_slope_MEAN
STD AwcHuc12_wtd_slope_STD
ZONE_CODE AwcHuc12_cat_slope_ZONE_CODE
COUNT AwcHuc12_cat_slope_COUNT
AREA AwcHuc12_cat_slope_AREA
MIN AwcHuc12_cat_slope_MIN
MAX AwcHuc12_cat_slope_MAX
RANGE AwcHuc12_cat_slope_RANGE
MEAN AwcHuc12_cat_slope_MEAN
STD AwcHuc12_cat_slope_STD
SUM AwcHuc12_cat_slope_SUM
MEDIAN AwcHuc12_cat_slope_MEDIAN
PCT90 AwcHuc12_cat_slope_PCT90


In [21]:
# # Export copies of dbf tables as csv
for table in outtables:
    tablename = arcpy.Describe(table).basename + ".csv"
    tablepath = os.path.join(outdir,tablename)
    print( tablepath)
    arcpy.conversion.TableToTable(table, outdir, tablename)

C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_awchuc12_wtd_north_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_awchuc12_cat_elev.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_awchuc12_wtd_elev.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_awchuc12_wtd_slope.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_awchuc12_cat_slope.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_awchuc12_wtd_wetland_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_awchuc12_wtd_glacier_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_awchuc12_wtd_lakepond_per.csv


In [22]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
# list to store covariate data frames
dfs = []

In [23]:
# Make catchment elev df
cat_df = pd.DataFrame()
cat_field_list = []
for field in arcpy.ListFields(cat_elev):
    cat_field_list.append(field.name)
cat_elev_arr = arcpy.da.TableToNumPyArray(cat_elev,cat_field_list)
cat_df = pd.DataFrame(cat_elev_arr)
cat_df = cat_df.drop(["OBJECTID","AwcHuc12_cat_elev_ZONE_CODE","region","cat_ID_txt"],axis=1)
cat_df = cat_df.set_index('cat_ID_con')
dfs.append(cat_df)
cat_df


,AwcHuc12_cat_elev_COUNT,AwcHuc12_cat_elev_AREA,AwcHuc12_cat_elev_MIN,AwcHuc12_cat_elev_MAX,AwcHuc12_cat_elev_RANGE,AwcHuc12_cat_elev_MEAN,AwcHuc12_cat_elev_STD,AwcHuc12_cat_elev_SUM,AwcHuc12_cat_elev_VARIETY,AwcHuc12_cat_elev_MAJORITY,AwcHuc12_cat_elev_MINORITY,AwcHuc12_cat_elev_MEDIAN,AwcHuc12_cat_elev_PCT90
cat_ID_con,,,,,,,,,,,,,
Prince_William_Sound_3038,3448.00,344800.00,0,96,96,10.59,10.72,36518.00,93,5,27,8,17
Prince_William_Sound_13227,297.00,29700.00,0,79,79,24.72,12.92,7342.00,60,14,1,22,37
Prince_William_Sound_17027,4338.00,433800.00,0,208,208,70.89,35.61,307536.00,201,48,0,61,119
Prince_William_Sound_17697,2137.00,213700.00,3,118,115,61.09,29.40,130546.00,116,3,25,60,101
Prince_William_Sound_18357,80.00,8000.00,4,12,8,6.31,1.91,505.00,9,5,12,5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Copper_River_75003900029086,1469.00,146900.00,2,7,5,3.88,0.89,5696.00,6,4,7,4,5
Copper_River_75003900044552,11111.00,1111100.00,3,225,222,75.24,57.47,836045.00,223,5,225,59,153
Copper_River_75003900054944,121.00,12100.00,3,5,2,4.11,0.79,497.00,3,5,3,4,5


In [24]:
# Make catchment slope df
cat_sl_df = pd.DataFrame()
cat_sl_field_list = []
for field in arcpy.ListFields(cat_slope):
    cat_sl_field_list.append(field.name)
cat_sl_arr = arcpy.da.TableToNumPyArray(cat_slope, cat_sl_field_list)
cat_sl_df = pd.DataFrame(cat_sl_arr)
cat_sl_df = cat_sl_df.drop(["OBJECTID", "AwcHuc12_cat_slope_ZONE_CODE","region","cat_ID_txt"],axis=1)
cat_sl_df = cat_sl_df.set_index('cat_ID_con')
dfs.append(cat_sl_df)
cat_sl_df

,AwcHuc12_cat_slope_COUNT,AwcHuc12_cat_slope_AREA,AwcHuc12_cat_slope_MIN,AwcHuc12_cat_slope_MAX,AwcHuc12_cat_slope_RANGE,AwcHuc12_cat_slope_MEAN,AwcHuc12_cat_slope_STD,AwcHuc12_cat_slope_SUM,AwcHuc12_cat_slope_MEDIAN,AwcHuc12_cat_slope_PCT90
cat_ID_con,,,,,,,,,,
Prince_William_Sound_3038,3448.00,344800.00,0.00,24.19,24.19,5.34,4.35,18396.22,4.04,11.39
Prince_William_Sound_13227,297.00,29700.00,0.00,22.82,22.82,8.72,3.92,2589.60,8.50,13.66
Prince_William_Sound_17027,4338.00,433800.00,0.00,37.53,37.53,11.51,6.66,49949.38,10.36,20.81
Prince_William_Sound_17697,2137.00,213700.00,0.00,38.09,38.09,14.63,7.43,31268.21,14.78,24.27
Prince_William_Sound_18357,80.00,8000.00,0.00,11.41,11.41,3.54,2.37,282.92,3.20,6.75
...,...,...,...,...,...,...,...,...,...,...
Copper_River_75003900029086,1469.00,146900.00,0.00,6.71,6.71,0.93,1.49,1367.85,0.00,3.37
Copper_River_75003900044552,11111.00,1111100.00,0.00,37.33,37.33,10.44,8.06,115974.51,8.11,22.92
Copper_River_75003900054944,121.00,12100.00,0.00,3.44,3.44,0.97,1.32,117.25,0.00,2.85


In [25]:
# Make watershed elev df
wtd_df = pd.DataFrame()
wtd_field_list = []
for field in arcpy.ListFields(wtd_elev):
    wtd_field_list.append(field.name)
wtd_elev_arr = arcpy.da.TableToNumPyArray(wtd_elev,wtd_field_list)
wtd_df = pd.DataFrame(wtd_elev_arr)
wtd_df = wtd_df.drop(["OBJECTID","NHDPlusID","cat_ID","region", "cat_ID_txt"],axis=1)
wtd_df = wtd_df.set_index('cat_ID_con')
dfs.append(wtd_df)
wtd_df

,AwcHuc12_wtd_elev_MIN,AwcHuc12_wtd_elev_MAX,AwcHuc12_wtd_elev_MEAN,AwcHuc12_wtd_elev_STD
cat_ID_con,,,,
Prince_William_Sound_3038,0,678,201.13,162.71
Prince_William_Sound_13227,0,580,223.81,125.00
Prince_William_Sound_17027,0,553,206.91,108.41
Prince_William_Sound_17697,3,692,286.20,140.33
Prince_William_Sound_18357,2,813,264.07,169.95
...,...,...,...,...
Copper_River_75003900029086,0,20,8.58,3.16
Copper_River_75003900044552,3,527,180.59,118.27
Copper_River_75003900054944,3,66,7.66,6.13


In [26]:
# Make watershed slope df
wtd_sl_df = pd.DataFrame()
wtd_sl_field_list = []
for field in arcpy.ListFields(wtd_slope):
    wtd_sl_field_list.append(field.name)
wtd_sl_arr = arcpy.da.TableToNumPyArray(wtd_slope, wtd_sl_field_list)
wtd_sl_df = pd.DataFrame(wtd_sl_arr)
wtd_sl_df = wtd_sl_df.drop(["OBJECTID","NHDPlusID","cat_ID","region","cat_ID","cat_ID_txt","NHDPlusID"],axis=1)
wtd_sl_df = wtd_sl_df.set_index('cat_ID_con')
dfs.append(wtd_sl_df)
wtd_sl_df

,AwcHuc12_wtd_slope_MIN,AwcHuc12_wtd_slope_MAX,AwcHuc12_wtd_slope_MEAN,AwcHuc12_wtd_slope_STD
cat_ID_con,,,,
Prince_William_Sound_3038,0.00,59.63,15.49,11.44
Prince_William_Sound_13227,0.00,57.61,23.72,10.73
Prince_William_Sound_17027,0.00,61.26,15.85,10.88
Prince_William_Sound_17697,0.00,63.16,18.06,11.76
Prince_William_Sound_18357,0.00,57.00,20.20,11.88
...,...,...,...,...
Copper_River_75003900029086,0.00,16.20,1.31,1.74
Copper_River_75003900044552,0.00,55.48,12.94,8.59
Copper_River_75003900054944,0.00,26.16,2.09,2.88


In [27]:
# Make watershed north df
wtd_n_df = pd.DataFrame()
wtd_n_field_list = []
for field in arcpy.ListFields(wtd_per_north):
    wtd_n_field_list.append(field.name)
wtd_n_arr = arcpy.da.TableToNumPyArray(wtd_per_north,wtd_n_field_list)
wtd_n_df = pd.DataFrame(wtd_n_arr)
wtd_n_df = wtd_n_df.drop(["OBJECTID","region","cat_ID_txt"],axis=1)
wtd_n_df = wtd_n_df.set_index('cat_ID_con')
dfs.append(wtd_n_df)
wtd_n_df

,AwcHuc12_non_north_area,AwcHuc12_north_area,AwcHuc12_wtd_north_per
cat_ID_con,,,
Prince_William_Sound_3038,24066600.00,8352000.00,25.76
Prince_William_Sound_13227,679900.00,627900.00,48.01
Prince_William_Sound_17027,2596000.00,1163500.00,30.95
Prince_William_Sound_17697,6874200.00,2365900.00,25.60
Prince_William_Sound_18357,15359800.00,4876600.00,24.10
...,...,...,...
Copper_River_75003900029086,17465000.00,1929700.00,9.95
Copper_River_75003900044552,10002100.00,2113300.00,17.44
Copper_River_75003900054944,7381300.00,1345600.00,15.42


In [28]:
# Make watershed wetland df
wtd_wet_df = pd.DataFrame()
wtd_wet_field_list = []
for field in arcpy.ListFields(wtd_wet):
    wtd_wet_field_list.append(field.name)
wtd_wet_arr = arcpy.da.TableToNumPyArray(wtd_wet,wtd_wet_field_list)
wtd_wet_df = pd.DataFrame(wtd_wet_arr)
wtd_wet_df = wtd_wet_df.drop(["OBJECTID","region","cat_ID_txt"],axis=1)
wtd_wet_df = wtd_wet_df.set_index('cat_ID_con')
dfs.append(wtd_wet_df)
wtd_wet_df

,AwcHuc12_non_wetland_area,AwcHuc12_wetland_area,AwcHuc12_wtd_wet_per
cat_ID_con,,,
Prince_William_Sound_3038,25232500.00,7186100.00,22.17
Prince_William_Sound_13227,1298800.00,9000.00,0.69
Prince_William_Sound_17027,3275800.00,483700.00,12.87
Prince_William_Sound_17697,8549500.00,690600.00,7.47
Prince_William_Sound_18357,19103600.00,1132800.00,5.60
...,...,...,...
Copper_River_75003900029086,318600.00,19076100.00,98.36
Copper_River_75003900044552,10373300.00,1742100.00,14.38
Copper_River_75003900054944,5613800.00,3113100.00,35.67


In [29]:
# Make watershed lakes df
wtd_lp_df = pd.DataFrame()
wtd_lp_field_list = []
for field in arcpy.ListFields(wtd_lp):
    wtd_lp_field_list.append(field.name)
wtd_lp_arr = arcpy.da.TableToNumPyArray(wtd_lp, wtd_lp_field_list)
wtd_lp_df = pd.DataFrame(wtd_lp_arr)
wtd_lp_df = wtd_lp_df.drop(["OBJECTID","region","cat_ID_txt","cat_ID","FType"],axis=1)
wtd_lp_df = wtd_lp_df.set_index('cat_ID_con')
dfs.append(wtd_lp_df)
wtd_lp_df

,AwcHuc12_wtd_lake_area_sqm,AwcHuc12_wtd_lake_per
cat_ID_con,,
Prince_William_Sound_17027,5688.95,0.15
Prince_William_Sound_17697,38507.61,0.42
Prince_William_Sound_18357,6904.33,0.03
Prince_William_Sound_18547,25771.29,0.11
Prince_William_Sound_18737,90355.70,0.14
...,...,...
Copper_River_75019800020292,1707529.72,0.54
Copper_River_75019800020572,5727440.86,0.61
Copper_River_75019800020746,7836889.55,1.75


In [30]:
# Make watershed glacier df
wtd_glac_df = pd.DataFrame()
wtd_glac_field_list = []
for field in arcpy.ListFields(wtd_glac):
    wtd_glac_field_list.append(field.name)
wtd_glac_arr = arcpy.da.TableToNumPyArray(wtd_glac, wtd_glac_field_list)
wtd_glac_df = pd.DataFrame(wtd_glac_arr)
wtd_glac_df = wtd_glac_df.drop(["OBJECTID","region","cat_ID_txt","cat_ID","O1Region"],axis=1)
wtd_glac_df = wtd_glac_df.set_index('cat_ID_con')
dfs.append(wtd_glac_df)
wtd_glac_df

,AwcHuc12_wtd_glacier_area_sqm,AwcHuc12_wtd_glacier_per
cat_ID_con,,
Prince_William_Sound_18357,255413.00,1.26
Prince_William_Sound_18547,83990.36,0.35
Prince_William_Sound_18737,866494.65,1.34
Prince_William_Sound_18747,866494.65,1.33
Prince_William_Sound_27856,176024.99,3.75
...,...,...
Copper_River_75019800018811,106387407.76,22.21
Copper_River_75019800019142,12957496.80,2.12
Copper_River_75019800019853,12957496.80,4.51



## Merge all covariate dataframes together and drop unnecessary columns
## No Longer using dfs to merge and Export - Instead Merge watershed polygons and join fields
 * Recalculate cat_ID as float64 type
 * Reorder columns
 * Export final csv


In [31]:
# Merge all data frames together
import numpy as np
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_con',how="outer"), dfs)

df_final.columns = list(uniquify(df_final))
#List of final columns in the order to output
# final_cols_old = ['cat_ID_txt','cat_ID','region', 'AwcHuc12_cat_slope_COUNT', 'AwcHuc12_cat_slope_AREA', 'AwcHuc12_cat_slope_MIN', 'AwcHuc12_cat_slope_MAX',
#               'AwcHuc12_cat_slope_RANGE','AwcHuc12_cat_slope_MEAN', 'AwcHuc12_cat_slope_STD', 'AwcHuc12_cat_slope_SUM', 'AwcHuc12_cat_slope_MEDIAN', 'AwcHuc12_cat_slope_PCT90',
#               'AwcHuc12_cat_elev_COUNT', 'AwcHuc12_cat_elev_AREA', 'AwcHuc12_cat_elev_MIN', 'AwcHuc12_cat_elev_MAX', 'AwcHuc12_cat_elev_RANGE', 'AwcHuc12_cat_elev_MEAN', 'AwcHuc12_cat_elev_STD',
#               'AwcHuc12_cat_elev_SUM', 'AwcHuc12_cat_elev_VARIETY', 'AwcHuc12_cat_elev_MAJORITY', 'AwcHuc12_cat_elev_MINORITY', 'AwcHuc12_cat_elev_MEDIAN', 'AwcHuc12_cat_elev_PCT90',
#               'AwcHuc12_wtd_elev_COUNT', 'AwcHuc12_wtd_elev_AREA', 'AwcHuc12_wtd_elev_MIN', 'AwcHuc12_wtd_elev_MAX', 'AwcHuc12_wtd_elev_RANGE', 'AwcHuc12_wtd_elev_MEAN',
#               'AwcHuc12_wtd_elev_STD', 'AwcHuc12_wtd_elev_SUM', 'AwcHuc12_wtd_elev_VARIETY', 'AwcHuc12_wtd_elev_MAJORITY', 'AwcHuc12_wtd_elev_MINORITY',
#               'AwcHuc12_wtd_elev_MEDIAN', 'AwcHuc12_wtd_elev_PCT90', 'AwcHuc12_wtd_slope_COUNT', 'AwcHuc12_wtd_slope_AREA', 'AwcHuc12_wtd_slope_MIN', 'AwcHuc12_wtd_slope_MAX',
#               'AwcHuc12_wtd_slope_RANGE', 'AwcHuc12_wtd_slope_MEAN', 'AwcHuc12_wtd_slope_STD', 'AwcHuc12_wtd_slope_SUM', 'AwcHuc12_wtd_slope_MEDIAN', 'AwcHuc12_wtd_slope_PCT90',
#               'AwcHuc12_non_north_area', 'AwcHuc12_north_area', 'AwcHuc12_wtd_north_per', 'non_wetland_area', 'AwcHuc12_wetland_area', 'AwcHuc12_wtd_wet_per',
#               'AwcHuc12_wtd_lake_area_sqm', 'AwcHuc12_wtd_lake_per', 'AwcHuc12_wtd_glacier_area_sqm', 'AwcHuc12_wtd_glacier_per' ]
final_cols = ['AwcHuc12_cat_slope_COUNT', 'AwcHuc12_cat_slope_AREA', 'AwcHuc12_cat_slope_MIN', 'AwcHuc12_cat_slope_MAX',
              'AwcHuc12_cat_slope_RANGE','AwcHuc12_cat_slope_MEAN', 'AwcHuc12_cat_slope_STD', 'AwcHuc12_cat_slope_SUM', 'AwcHuc12_cat_slope_MEDIAN', 'AwcHuc12_cat_slope_PCT90',
              'AwcHuc12_cat_elev_COUNT', 'AwcHuc12_cat_elev_AREA', 'AwcHuc12_cat_elev_MIN', 'AwcHuc12_cat_elev_MAX', 'AwcHuc12_cat_elev_RANGE', 'AwcHuc12_cat_elev_MEAN', 'AwcHuc12_cat_elev_STD',
              'AwcHuc12_cat_elev_SUM', 'AwcHuc12_cat_elev_VARIETY', 'AwcHuc12_cat_elev_MAJORITY', 'AwcHuc12_cat_elev_MINORITY', 'AwcHuc12_cat_elev_MEDIAN', 'AwcHuc12_cat_elev_PCT90',
              'AwcHuc12_wtd_elev_MIN', 'AwcHuc12_wtd_elev_MAX','AwcHuc12_wtd_elev_MEAN','AwcHuc12_wtd_elev_STD','AwcHuc12_wtd_slope_MIN', 'AwcHuc12_wtd_slope_MAX','AwcHuc12_wtd_slope_MEAN',
              'AwcHuc12_wtd_slope_STD','AwcHuc12_non_north_area', 'AwcHuc12_north_area', 'AwcHuc12_wtd_north_per', 'non_wetland_area', 'AwcHuc12_wetland_area', 'AwcHuc12_wtd_wet_per',
              'AwcHuc12_wtd_lake_area_sqm', 'AwcHuc12_wtd_lake_per', 'AwcHuc12_wtd_glacier_area_sqm', 'AwcHuc12_wtd_glacier_per' ]

# # Export merged dataframe to csv
#cov_csv_out = os.path.join(outdir,'AKSSF_AWC_HUC12s_Covariates.csv')
#df_final.to_csv(cov_csv_out, encoding = 'utf-8')
#print('Export all covariates dataframe to csv complete')

df_final

,AwcHuc12_cat_elev_COUNT,AwcHuc12_cat_elev_AREA,AwcHuc12_cat_elev_MIN,AwcHuc12_cat_elev_MAX,AwcHuc12_cat_elev_RANGE,AwcHuc12_cat_elev_MEAN,AwcHuc12_cat_elev_STD,AwcHuc12_cat_elev_SUM,AwcHuc12_cat_elev_VARIETY,AwcHuc12_cat_elev_MAJORITY,...,AwcHuc12_non_north_area,AwcHuc12_north_area,AwcHuc12_wtd_north_per,AwcHuc12_non_wetland_area,AwcHuc12_wetland_area,AwcHuc12_wtd_wet_per,AwcHuc12_wtd_lake_area_sqm,AwcHuc12_wtd_lake_per,AwcHuc12_wtd_glacier_area_sqm,AwcHuc12_wtd_glacier_per
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Prince_William_Sound_3038,3448.00,344800.00,0,96,96,10.59,10.72,36518.00,93,5,...,24066600.00,8352000.00,25.76,25232500.00,7186100.00,22.17,61494.14,0.19,NaN,NaN
Prince_William_Sound_13227,297.00,29700.00,0,79,79,24.72,12.92,7342.00,60,14,...,679900.00,627900.00,48.01,1298800.00,9000.00,0.69,NaN,NaN,NaN,NaN
Prince_William_Sound_17027,4338.00,433800.00,0,208,208,70.89,35.61,307536.00,201,48,...,2596000.00,1163500.00,30.95,3275800.00,483700.00,12.87,5688.95,0.15,NaN,NaN
Prince_William_Sound_17697,2137.00,213700.00,3,118,115,61.09,29.40,130546.00,116,3,...,6874200.00,2365900.00,25.60,8549500.00,690600.00,7.47,38507.61,0.42,NaN,NaN
Prince_William_Sound_18357,80.00,8000.00,4,12,8,6.31,1.91,505.00,9,5,...,15359800.00,4876600.00,24.10,19103600.00,1132800.00,5.60,6904.33,0.03,255413.00,1.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Copper_River_75003900029086,1469.00,146900.00,2,7,5,3.88,0.89,5696.00,6,4,...,17465000.00,1929700.00,9.95,318600.00,19076100.00,98.36,576851.52,2.96,NaN,NaN
Copper_River_75003900044552,11111.00,1111100.00,3,225,222,75.24,57.47,836045.00,223,5,...,10002100.00,2113300.00,17.44,10373300.00,1742100.00,14.38,424104.69,3.49,NaN,NaN
Copper_River_75003900054944,121.00,12100.00,3,5,2,4.11,0.79,497.00,3,5,...,7381300.00,1345600.00,15.42,5613800.00,3113100.00,35.67,148894.93,1.70,NaN,NaN



## Merge watersheds and join covariates for data explore in GIS
* Collect all _wtds_merge polygons and merge together
* Join Covariates using cat_ID_con field


In [32]:
arcpy.env.workspace = outgdb
outtables.append(lcld_table)
# Merge watersheds
wtds = [fc for fc in arcpy.ListFeatureClasses('*_wtds_merge','POLYGON')]
akssf_wtds = arcpy.Merge_management(wtds,'AKSSF_AwcHuc12_Watersheds_Merge')
dropfields = ['OBJECTID','cat_ID_con', 'cat_ID', 'cat_ID_txt', 'region','O1Region','FType','NHDPlusID']
for table in outtables:
    tname = table.getOutput(0).split("\\")[-1]
    fields =[f.name for f in arcpy.ListFields(tname) if f.name not in dropfields]
    print(f'Joining fields : {fields} to all Watersheds\n{("*"*100)}')
    arcpy.JoinField_management(akssf_wtds,'cat_ID_con',table, 'cat_ID_con', fields)
akssf_wtds_table = arcpy.TableToTable_conversion(akssf_wtds,outgdb,'AKSSF_AwcHuc12_Covariates')

Joining fields : ['AwcHuc12_non_north_area', 'AwcHuc12_north_area', 'AwcHuc12_wtd_north_per'] to all Watersheds
****************************************************************************************************
Joining fields : ['AwcHuc12_cat_elev_ZONE_CODE', 'AwcHuc12_cat_elev_COUNT', 'AwcHuc12_cat_elev_AREA', 'AwcHuc12_cat_elev_MIN', 'AwcHuc12_cat_elev_MAX', 'AwcHuc12_cat_elev_RANGE', 'AwcHuc12_cat_elev_MEAN', 'AwcHuc12_cat_elev_STD', 'AwcHuc12_cat_elev_SUM', 'AwcHuc12_cat_elev_VARIETY', 'AwcHuc12_cat_elev_MAJORITY', 'AwcHuc12_cat_elev_MINORITY', 'AwcHuc12_cat_elev_MEDIAN', 'AwcHuc12_cat_elev_PCT90'] to all Watersheds
****************************************************************************************************
Joining fields : ['AwcHuc12_wtd_elev_MIN', 'AwcHuc12_wtd_elev_MAX', 'AwcHuc12_wtd_elev_MEAN', 'AwcHuc12_wtd_elev_STD'] to all Watersheds
****************************************************************************************************
Joining fields : ['AwcHuc12_wtd

In [33]:
# Export watershed as table after join fields
akkssf_cov_csv = arcpy.conversion.TableToTable(akssf_wtds_table, outdir, 'AKSSF_AWC_HUC12s_Covariates.csv')
print('Export all covariates dataframe to csv complete')


Export all covariates dataframe to csv complete


In [34]:
import pandas as pd
import numpy as np

#convert to df and check
wtds_df = pd.DataFrame()
wtds_fl = []

for field in arcpy.ListFields(akssf_wtds_table):
    wtds_fl.append(field.name)
akssf_wtds_arr = arcpy.da.TableToNumPyArray(akssf_wtds_table, wtds_fl)
akssf_wtds_arr
wtds_df = pd.DataFrame(akssf_wtds_arr)
wtds_df = wtds_df.set_index('cat_ID_con')
wtds_df


,OBJECTID,ORIG_FID,MERGE_SRC,cat_ID,cat_ID_txt,NHDPlusID,Shape_Length,Shape_Area,AwcHuc12_non_north_area,AwcHuc12_north_area,...,AwcHuc12_wtd_lcld_mn_2010,AwcHuc12_wtd_lcld_mn_2011,AwcHuc12_wtd_lcld_mn_2012,AwcHuc12_wtd_lcld_mn_2013,AwcHuc12_wtd_lcld_mn_2014,AwcHuc12_wtd_lcld_mn_2015,AwcHuc12_wtd_lcld_mn_2016,AwcHuc12_wtd_lcld_mn_2017,AwcHuc12_wtd_lcld_mn_2018,AwcHuc12_wtd_lcld_mn_2019
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Cook_Inlet_75004200000901,1,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200000901.00,75004200000901,75004200000901.00,46480.29,36962575.23,31878600.00,5081600.00,...,529.25,523.44,535.94,528.98,502.93,513.58,520.67,519.30,520.23,519.21
Cook_Inlet_75004200001724,2,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200001724.00,75004200001724,75004200001724.00,118239.51,170812175.08,126608700.00,44202600.00,...,528.82,525.15,536.58,529.06,507.13,508.44,518.90,518.61,519.17,515.67
Cook_Inlet_75004200001726,3,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200001726.00,75004200001726,75004200001726.00,232595.15,570114975.40,439802800.00,130311400.00,...,530.27,526.21,538.68,530.11,510.24,511.44,523.11,521.35,523.30,518.68
Cook_Inlet_75004200001493,4,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200001493.00,75004200001493,75004200001493.00,19800.68,9640599.99,8601100.00,1039700.00,...,542.38,518.18,545.47,525.08,495.20,507.19,521.56,519.55,523.21,516.99
Cook_Inlet_75004200004105,5,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200004105.00,75004200004105,75004200004105.00,21044.04,14023024.97,12141000.00,1882800.00,...,550.69,536.13,553.18,536.80,507.73,515.97,524.91,529.52,529.46,517.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Prince_William_Sound_91741,1014,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,91741.00,91741,NaN,220440.00,572857300.00,386812700.00,186044600.00,...,543.46,541.08,557.10,548.00,540.37,537.09,540.25,536.80,545.89,536.36
Prince_William_Sound_91751,1015,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,91751.00,91751,NaN,220040.00,574793800.00,388594100.00,186199700.00,...,543.40,541.02,557.05,547.95,540.33,537.04,540.19,536.74,545.85,536.31
Prince_William_Sound_91921,1016,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,91921.00,91921,NaN,233240.00,739347900.00,506926400.00,232421500.00,...,541.73,539.30,556.09,546.72,538.25,535.18,537.83,535.30,544.02,534.66


In [35]:
# bbay_df = df_final.filter(like='Bristol_Bay', axis = 0)
# bbay_df

In [36]:
# # kod_df = df_final.filter(like='Kodiak', axis = 0)
# kod_df

In [37]:
pws_df = wtds_df.filter(like='Prince', axis = 0)
pws_df

,OBJECTID,ORIG_FID,MERGE_SRC,cat_ID,cat_ID_txt,NHDPlusID,Shape_Length,Shape_Area,AwcHuc12_non_north_area,AwcHuc12_north_area,...,AwcHuc12_wtd_lcld_mn_2010,AwcHuc12_wtd_lcld_mn_2011,AwcHuc12_wtd_lcld_mn_2012,AwcHuc12_wtd_lcld_mn_2013,AwcHuc12_wtd_lcld_mn_2014,AwcHuc12_wtd_lcld_mn_2015,AwcHuc12_wtd_lcld_mn_2016,AwcHuc12_wtd_lcld_mn_2017,AwcHuc12_wtd_lcld_mn_2018,AwcHuc12_wtd_lcld_mn_2019
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Prince_William_Sound_3038,907,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,3038.00,3038,NaN,34060.00,32418600.00,24066600.00,8352000.00,...,504.25,497.99,519.37,516.44,487.40,459.34,438.09,491.19,501.66,458.89
Prince_William_Sound_13227,908,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,13227.00,13227,NaN,6620.00,1307800.00,679900.00,627900.00,...,506.24,509.96,533.76,524.00,488.00,474.74,469.88,485.92,492.00,486.10
Prince_William_Sound_17027,909,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,17027.00,17027,NaN,12880.00,3759500.00,2596000.00,1163500.00,...,510.35,500.02,530.86,524.14,492.49,453.12,442.00,504.22,503.26,485.73
Prince_William_Sound_17697,910,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,17697.00,17697,NaN,17020.00,9240100.00,6874200.00,2365900.00,...,522.74,524.49,548.31,533.97,502.49,482.51,475.55,515.76,506.68,502.77
Prince_William_Sound_18357,911,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,18357.00,18357,NaN,26060.00,20236400.00,15359800.00,4876600.00,...,516.34,510.73,538.70,528.86,496.91,461.28,462.00,506.69,496.49,477.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Prince_William_Sound_91741,1014,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,91741.00,91741,NaN,220440.00,572857300.00,386812700.00,186044600.00,...,543.46,541.08,557.10,548.00,540.37,537.09,540.25,536.80,545.89,536.36
Prince_William_Sound_91751,1015,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,91751.00,91751,NaN,220040.00,574793800.00,388594100.00,186199700.00,...,543.40,541.02,557.05,547.95,540.33,537.04,540.19,536.74,545.85,536.31
Prince_William_Sound_91921,1016,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,91921.00,91921,NaN,233240.00,739347900.00,506926400.00,232421500.00,...,541.73,539.30,556.09,546.72,538.25,535.18,537.83,535.30,544.02,534.66


In [38]:
# ci_df = df_final.filter(like='Cook', axis = 0)
# ci_df

In [39]:
cop_df = wtds_df.filter(like='Copper', axis = 0)
cop_df

,OBJECTID,ORIG_FID,MERGE_SRC,cat_ID,cat_ID_txt,NHDPlusID,Shape_Length,Shape_Area,AwcHuc12_non_north_area,AwcHuc12_north_area,...,AwcHuc12_wtd_lcld_mn_2010,AwcHuc12_wtd_lcld_mn_2011,AwcHuc12_wtd_lcld_mn_2012,AwcHuc12_wtd_lcld_mn_2013,AwcHuc12_wtd_lcld_mn_2014,AwcHuc12_wtd_lcld_mn_2015,AwcHuc12_wtd_lcld_mn_2016,AwcHuc12_wtd_lcld_mn_2017,AwcHuc12_wtd_lcld_mn_2018,AwcHuc12_wtd_lcld_mn_2019
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Copper_River_75019800007852,662,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75019800007852.00,75019800007852,75019800007852.00,62860.01,71446424.90,55997900.00,15343900.00,...,505.76,509.36,517.00,528.19,522.83,505.11,506.70,501.25,525.72,502.57
Copper_River_75019800005498,663,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75019800005498.00,75019800005498,75019800005498.00,113120.00,244654125.27,218281700.00,26206700.00,...,536.89,539.65,551.99,555.19,552.80,543.47,546.10,536.22,552.19,540.93
Copper_River_75019800019853,664,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75019800019853.00,75019800019853,75019800019853.00,145060.01,287506175.05,231029100.00,56257700.00,...,506.71,514.94,526.55,533.24,519.68,513.88,511.48,504.33,533.34,511.44
Copper_River_75019800002596,665,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75019800002596.00,75019800002596,75019800002596.00,79560.00,94152599.99,66925300.00,27116100.00,...,501.23,509.12,518.01,525.45,504.54,501.97,499.07,496.37,520.34,501.54
Copper_River_75019800019678,666,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75019800019678.00,75019800019678,75019800019678.00,77840.01,126024425.03,101165100.00,24739500.00,...,496.92,504.07,512.62,519.63,499.95,497.23,487.87,488.32,511.41,492.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Copper_River_75003900029086,902,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75003900029086.00,75003900029086,75003900029086.00,48830.00,19478424.97,17465000.00,1929700.00,...,472.78,458.79,485.64,487.82,423.33,414.76,410.16,470.74,451.56,440.00
Copper_River_75003900044552,903,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75003900044552.00,75003900044552,75003900044552.00,26460.00,12151350.03,10002100.00,2113300.00,...,505.85,494.67,529.90,514.89,479.22,414.40,425.84,496.15,491.61,470.63
Copper_River_75003900054944,904,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75003900054944.00,75003900054944,75003900054944.00,24720.00,8759500.00,7381300.00,1345600.00,...,475.44,476.38,505.54,493.38,470.74,423.43,413.12,478.51,472.77,434.10


In [40]:
ci_df = wtds_df.filter(like='Cook', axis = 0)
ci_df

,OBJECTID,ORIG_FID,MERGE_SRC,cat_ID,cat_ID_txt,NHDPlusID,Shape_Length,Shape_Area,AwcHuc12_non_north_area,AwcHuc12_north_area,...,AwcHuc12_wtd_lcld_mn_2010,AwcHuc12_wtd_lcld_mn_2011,AwcHuc12_wtd_lcld_mn_2012,AwcHuc12_wtd_lcld_mn_2013,AwcHuc12_wtd_lcld_mn_2014,AwcHuc12_wtd_lcld_mn_2015,AwcHuc12_wtd_lcld_mn_2016,AwcHuc12_wtd_lcld_mn_2017,AwcHuc12_wtd_lcld_mn_2018,AwcHuc12_wtd_lcld_mn_2019
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Cook_Inlet_75004200000901,1,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200000901.00,75004200000901,75004200000901.00,46480.29,36962575.23,31878600.00,5081600.00,...,529.25,523.44,535.94,528.98,502.93,513.58,520.67,519.30,520.23,519.21
Cook_Inlet_75004200001724,2,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200001724.00,75004200001724,75004200001724.00,118239.51,170812175.08,126608700.00,44202600.00,...,528.82,525.15,536.58,529.06,507.13,508.44,518.90,518.61,519.17,515.67
Cook_Inlet_75004200001726,3,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200001726.00,75004200001726,75004200001726.00,232595.15,570114975.40,439802800.00,130311400.00,...,530.27,526.21,538.68,530.11,510.24,511.44,523.11,521.35,523.30,518.68
Cook_Inlet_75004200001493,4,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200001493.00,75004200001493,75004200001493.00,19800.68,9640599.99,8601100.00,1039700.00,...,542.38,518.18,545.47,525.08,495.20,507.19,521.56,519.55,523.21,516.99
Cook_Inlet_75004200004105,5,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75004200004105.00,75004200004105,75004200004105.00,21044.04,14023024.97,12141000.00,1882800.00,...,550.69,536.13,553.18,536.80,507.73,515.97,524.91,529.52,529.46,517.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cook_Inlet_75005400000004,657,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75005400000004.00,75005400000004,75005400000004.00,94696.49,97406500.13,84065900.00,13340300.00,...,551.60,543.33,549.45,543.40,532.86,537.47,540.13,532.93,545.78,538.48
Cook_Inlet_75005400001431,658,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75005400001431.00,75005400001431,75005400001431.00,137314.80,299222899.80,236668000.00,62550200.00,...,528.10,516.80,525.54,525.83,513.12,514.19,521.02,507.35,516.73,506.96
Cook_Inlet_75005400031008,659,1,D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb...,75005400031008.00,75005400031008,75005400031008.00,133650.08,249792949.90,189522100.00,60275000.00,...,529.05,511.61,536.93,523.49,506.35,509.20,517.90,509.16,512.02,505.04
